In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import square
from scipy.special import diric
import matplotlib.colors as colors
import scipy.signal as signal
import sympy.integrals.transforms as transforms
from sympy import *
from sympy.solvers.solvers import solve
from IPython.display import display, Math
from matplotlib.patches import Circle
plt.rcParams.update({
    "text.usetex" : True,
    "text.latex.preamble" : r"\usepackage{amssymb} \usepackage{amsmath} \usepackage{faktor} \newcommand{\im}{\mathrm{j}} \newcommand{\e}{\mathrm{e}}",
    "font.size" : 11,
})

In [ ]:
def rect_impulse(t,Th):
    x = np.zeros_like(t)
    i = np.where((t >= -Th/2) & (t <= Th/2))
    x[i] = 1
    return x

def tri_impulse(t,Th):
    x = np.zeros_like(t)
    i = np.where((t >= -Th/2) & (t <0))
    x[i] = 2/Th*t[i]+1
    i = np.where((t <= +Th/2) & (t >= 0))
    x[i] = -2/Th*t[i]+1
    return x

def triangular_wave(t,T,Th,A):
    t = np.mod(t,T)
    x = np.zeros_like(t)
    i = np.where(t <= Th/2)
    x[i] = -2*A/Th*t[i]+A
    i = np.where(t >= T-Th/2)
    x[i] = 2*A/Th*t[i]+A*(1-2*T/Th)
    return x

---
header-includes:
   - \usepackage{amssymb}
   - \usepackage{trfsigns}
---

$\newcommand{\e}{\mathrm{e}}$
$\newcommand{\im}{\mathrm{j}}$
$\newcommand{\sinc}{\mathrm{sinc}}$
### Fourierkoeffizienten Rechteckschwingung
\begin{gather}
	X_0=2\int_0^{\frac{T_h}{2}}A\mathrm{d}t 
= 2At\Bigg |_{t=0}^{t=\frac{T_h}{2}}=A\cdot T_h\\
	X_\mu=\int_{-\frac{T_h}{2}}^{\frac{T_h}{2}}A\e^{-\im k \omega_0 t}\mathrm{d}t=\frac{A}{\im k \omega_0}\left(\e^{\im \pi  \frac{T_h}{T}}-\e^{-\im \pi \mu \frac{T_h}{T}}\right)\\
	=\frac{AT}{\im \mu 2\pi }\left(\e^{\im \pi \mu \frac{T_h}{T}}-\e^{-\im \pi \mu \frac{T_h}{T}}\right)
	=\frac{AT}{\mu\pi}\sin(\pi \mu \frac{T_h}{T})\\
	=\frac{AT}{\mu\pi}\frac{T_h}{T_h}\sin(\pi \mu \frac{T_h}{T})={AT_h}\frac{\sin(\pi \mu \frac{T_h}{T})}{\pi \mu \frac{T_h}{T}}=A\cdot T_h\mathrm{sinc}(\pi \mu \frac{T_h}{T})
\end{gather}
### Faltung von $x(t)$ mit $y(t)$
\begin{gather}
	x(t)\ast y(t)=\begin{cases}
		\int_{t-T_h}^{-T+\frac{T_h}{2}}\frac{A^2}{2}\mathrm{d}\tau=\frac{A^2}{2}\left(-T+\frac{T_h}{2}-t+T_h\right)=-\frac{A^2}{2}\left(t+T-3\frac{T_h}{2}\right)&,-\frac{T}{2}\leq t \leq -\frac{T_h}{2}\\
		\int_{-\frac{T_h}{2}}^{t}\frac{A^2}{2}\mathrm{d}\tau=\frac{A^2}{2}\left(t+\frac{T_h}{2}\right)&,\frac{-T_h}{2}\leq t \leq \frac{T_h}{2} \\
		\int_{t-T_h}^{\frac{T_h}{2}}\frac{A^2}{2}\mathrm{d}\tau=\frac{A^2}{2}\left(\frac{T_h}{2}-t+T_h\right)=-\frac{A^2}{2}\left(t-3\frac{T_h}{2}\right)&,\frac{T_h}{2}\leq t \leq \frac{T}{2}
	\end{cases}
\end{gather}

In [ ]:
#fourier series of square wave, time shifted square wave and convolution between these two
T = 2.5
Th = T/2
A = 1.5
color1 = 'black'
color2 = 'C0'
color3 = 'C2'
color4 = 'C3'
color5 = 'C4'

fig = plt.figure(figsize=(9,7))


ax = fig.add_subplot(311)

t = np.arange(-5,5+1/1000,1/1000)
k = np.arange(-500,501)
kk, tt = np.meshgrid(k,t)
w = np.exp(1j*kk*tt*2*np.pi/T)


x = A*(square((t+Th/2)/T*2*np.pi,duty=Th/T)+1)/2
y= A/2*(square((t)/T*2*np.pi,duty=Th/T)+1)/2
z = A*Th*np.sinc(kk*Th/T)*1/2*A*Th*np.sinc(kk*Th/T)*np.exp(-1j*2*np.pi/T*kk*Th/2)*w
z = np.reshape(z, (t.size,k.size))
z = 1/T*np.sum(z,axis=1)
z = np.real(z)

ax.plot(t,x,color=color1,label='$x(t)$',clip_on=False)
ax.plot(t,y,color=color2,label=r'$y(t)=\frac{1}{2}x\left(t-\frac{T_h}{2}\right)$',clip_on=False)
ax.plot(t,z,color=color3,label=r'$z(t)=x(t)\ast y(t)$',clip_on=False)
ax.legend(loc='upper right')



ax.set_xlim(-5,5)
ax.set_xticks(np.arange(-5,6,1))

ax.set_xticks([-2*T,-T,-Th/2,0,Th/2,T,2*T])
ax.set_xticklabels(labels=['$-2T_0$','$-T_0$',r'$-\faktor{T_h}{2}$','$0$',r'$+\faktor{T_h}{2}$','$+T_0$','$+2T_0$'])
ax.set_xlabel('$t$')

ax.set_ylim(0,A)
ax.set_yticks(np.arange(0,A+A/2,A/2))
ax.set_yticklabels(['$0$',r'$\faktor{A}{2}$','$A$'])
ax.set_title(r'$x(t)=\mathrm{rect}\left(\frac{t}{T_h}\right),\quad\quad x(t)=x(t+T_0),\quad\quad 0<\frac{T_h}{T}\leq 1$',pad=20)
ax.grid(True)


k = np.arange(-5,6,1)
X = A*Th*np.sinc(k*Th/T)
Y = X*np.exp(-2j*np.pi/T*k*Th/2)/2
Z = X*Y
X_ph = np.angle(X)
i = np.where(k >0,1,0)
j = np.where(X_ph==np.pi,1,0)
i = np.logical_and(i,j)
i, = np.where(i==True)
X_ph[i] = -np.pi

ax1 = fig.add_subplot(334)
ax2 = fig.add_subplot(335)
ax3 = fig.add_subplot(336)
ax4 = fig.add_subplot(337)
ax5 = fig.add_subplot(338)
ax6 = fig.add_subplot(339)

ax1.set_xlabel('$\mu$')
ax2.set_xlabel('$\mu$')
ax3.set_xlabel('$\mu$')
ax4.set_xlabel('$\mu$')
ax5.set_xlabel('$\mu$')
ax6.set_xlabel('$\mu$')

ax1.plot(k,np.abs(X),marker='o',ls='',mec=color1,mfc=color1,
         label=r'$\left|\tilde{X}\left[\mu\right]\right|$',
         clip_on=False)
ax2.plot(k,np.abs(Y),marker='o',ls='',mec=color1,mfc=color2,
         label=r'$\left|\tilde{Y}\left[\mu\right]\right|$',
         clip_on=False)
ax3.plot(k,np.abs(Z),marker='o',ls='',mec=color1,mfc=color3,
         label=r'$\left|\tilde{Z}\left[\mu\right]\right|$',
         clip_on=False)

ax4.plot(k,X_ph,marker='o',ls='',mec=color1,mfc=color1,
         label=r'$\varphi\left(\tilde{X}\left[\mu\right]\right)$',
         clip_on=False)
ax5.plot(k,np.angle(Y),marker='o',ls='',mec=color1,mfc=color2,
         label=r'$\varphi\left(\tilde{Y}\left[\mu\right]\right)$',
         clip_on=False)
ax6.plot(k,np.angle(Z),marker='o',ls='',mec=color1,mfc=color3,
         label=r'$\varphi\left(\tilde{Z}\left[\mu\right]\right)$',
         clip_on=False)

# gestrichelte Linie zeichnen

k = np.arange(-5,5+1/1000,1/1000)
X = A*Th*np.sinc(k*Th/T)
Y = X*np.exp(-2j*np.pi/T*k*Th/2)/2
Z = X*Y
X_ph = np.angle(X)
i = np.where(k>0,1,10)
j = np.where(X_ph==np.pi,1,0)
i = np.logical_and(i,j)
i, = np.where(i == True)
X_ph[i] *= (-1)*np.sign(k)[i]


ax1.plot(k,np.abs(X),ls=':',color=color1,clip_on=False)
ax2.plot(k,np.abs(Y),ls=':',color=color2,clip_on=False)
ax3.plot(k,np.abs(Z),ls=':',color=color3,clip_on=False)

ax4.plot(k,X_ph,ls=':',color=color1,clip_on=False)
ax5.plot(k,np.angle(Y),ls=':',color=color2,clip_on=False)
ax6.plot(k,np.angle(Z),ls=':',color=color3,clip_on=False)

xticks = np.arange(-5,5+1,1)
yticks = np.arange(-np.pi,np.pi+np.pi/2,np.pi/2)
labels = ['$-\pi$',r'$-\faktor{\pi}{2}$','$0$',r'$+\faktor{\pi}{2}$',r'$+\pi$']

ax1.set_xlim(-5,5)
ax2.set_xlim(-5,5)
ax3.set_xlim(-5,5)
ax4.set_xlim(-5,5)
ax5.set_xlim(-5,5)
ax6.set_xlim(-5,5)

ax1.set_xticks(xticks)
ax2.set_xticks(xticks)
ax3.set_xticks(xticks)
ax4.set_xticks(xticks)
ax5.set_xticks(xticks)
ax6.set_xticks(xticks)



ax1.set_ylim(0,np.amax(np.abs(X)))
ax2.set_ylim(0,np.amax(np.abs(Y)))
ax3.set_ylim(0,np.amax(np.abs(Z)))

ax1.set_yticks(np.arange(0,A*Th+A*Th/2,A*Th/2))
ax1.set_yticklabels(['0',r'$\faktor{AT_h}{2}$',
                     '$A T_h$'])

ax2.set_yticks(np.arange(0,A*Th/2+A*Th/4,A*Th/4))
ax2.set_yticklabels(['0',r'$\faktor{A T_h}{4}$',
                     r'$\faktor{A T_h}{2}$'])
ax3.set_yticks(np.arange(0,A**2*Th**2/2+A**2*Th**2/4,A**2*Th**2/4))
ax3.set_yticklabels(['0',r'$\faktor{A^2 T_h^2}{4}$',
                     r'$\faktor{A^2 T_h^2}{2}$'])

ax4.set_ylim(-np.pi,+np.pi)
ax5.set_ylim(-np.pi,+np.pi)
ax6.set_ylim(-np.pi,+np.pi)

ax4.set_yticks(yticks)
ax5.set_yticks(yticks)
ax6.set_yticks(yticks)

ax4.set_yticklabels(labels)
ax5.set_yticklabels(labels)
ax6.set_yticklabels(labels)


ax1.legend(fontsize=12,loc='center right')
ax2.legend(fontsize=12,loc='center right')
ax3.legend(fontsize=12,loc='center right')
ax4.legend(fontsize=12,loc='upper right')
ax5.legend(fontsize=12,loc='upper right')
ax6.legend(fontsize=12,loc='upper right')

ax1.grid(True)
ax2.grid(True)
ax3.grid(True)
ax4.grid(True)
ax5.grid(True)
ax6.grid(True)

fig.tight_layout()
fig.savefig('fourier_series_rect.pdf')

$\newcommand{\e}{\mathrm{e}}$
$\newcommand{\im}{\mathrm{j}}$
$\newcommand{\sinc}{\mathrm{sinc}}$
### Fourierkoeffizienten Dreieckschwingung

\begin{gather}
	X_\mu = \int_{-\frac{T_h}{2}}^{+\frac{T_h}{2}}x(t)\e^{-\im \mu \omega t}\mathrm{d}t=\int_{-\frac{T_h}{2}}^0\left(\frac{2A}{T_h}t+A\right)\e^{-\im \mu \omega t}\mathrm{d}t+\int_0^{+\frac{T_h}{2}}\left(-\frac{2A}{T_h}t+A\right)\e^{-\im \mu \omega t}\mathrm{d}t\\
	=\frac{\left(\frac{2A}{T_h}t+A\right)}{-\im \mu \omega }\e^{-\im \mu \omega t}\Bigg|_{t=-\frac{T_h}{2}}^{0}+\int_{-\frac{T_h}{2}}^0\frac{\left(\frac{2A}{T_h}\right)}{\im \mu \omega }\e^{-\im \mu \omega t}\mathrm{d}t+\frac{\left(-\frac{2A}{T_h}t+A\right)}{-\im \mu \omega }\e^{-\im \mu \omega t}\Bigg|_{t=0}^{+\frac{T_h}{2}}-\int_0^{+\frac{T_h}{2}}\frac{\left(\frac{2A}{T_h}\right)}{\im \mu \omega }\e^{-\im \mu \omega t}\mathrm{d}t\\
	=-\frac{A}{\im \mu \omega }+\frac{\left(\frac{2A}{T_h}\right)}{\mu^2\omega^2}\e^{-\im \mu \omega t}\Bigg |_{t=-\frac{T_h}{2}}^{t=0}+\frac{A}{\im \mu \omega}+\frac{\left(\frac{2A}{T_h}\right)}{\mu^2\omega^2}\e^{-\im \mu \omega t}\Bigg |_{t=+\frac{T_h}{2}}^{t=0}\\
	=\frac{\left(\frac{2A}{T_h}\right)}{\mu^2\omega^2}\left[2-\e^{\im \mu \omega \frac{T_h}{2}}-\e^{-\im \mu \omega \frac{T_h}{2}}\right]=\frac{\left(\frac{4A}{T_h}\right)}{\mu^2\omega^2}\left[1-\cos(\mu\pi\frac{T_h}{T})\right]
\end{gather}
Es gilt (S. 83 in I. Bronshtein, K. Semendyayev, G. Musiol, and H. Mühlig, Handbook of
Mathematics, 6th ed. Springer Berlin, Heidelberg, 2015.):
\begin{gather}
	\sin^2(\alpha)=\frac{1}{2}(1-\cos(2\alpha))
\end{gather}
Also:
\begin{gather}
	\frac{\left(\frac{4A}{T_h}\right)}{\mu^2\omega^2}\left[1-\cos(\mu\pi\frac{T_h}{T})\right]=\frac{\left(\frac{4AT^2}{T_h}\right)}{\mu^24\pi^2}\left[1-\cos(\mu\pi\frac{T_h}{T})\right]=\frac{2AT^2}{T_h\mu^2\pi^2}\cdot\frac{1}{2}\left[1-\cos\left(2\mu\pi\frac{T_h}{2T}\right)\right]\\
	=\frac{2AT^2}{T_h\mu^2\pi^2}\left[\sin^2\left(\mu\pi\frac{T_h}{2T}\right)\right]=\frac{T_hA}{2}\frac{4T^2}{T_h^2\mu^2\pi^2}\left[\sin^2\left(\mu\pi\frac{T_h}{2T}\right)\right]=\frac{A\cdot T_h}{2}\sinc^2\left(\mu\pi\frac{T_h}{2T}\right)
\end{gather}

In [ ]:
T = 2.5
Th = T/2
A = 1.5
color1 = 'black'
color2 = 'C0'
color3 = 'C2'
color4 = 'C3'
color5 = 'C4'

fig = plt.figure(figsize=(9,7))

k = np.arange(-500,501)
        
ax = fig.add_subplot(311)
ax.set_xlabel('$t$')

x = triangular_wave(t=t,T=T,Th=Th,A=A)
y = triangular_wave(t=t-Th/2,T=T,Th=Th,A=A)/2

z = A*Th/2*np.sinc(kk*Th/(2*T))**2*1/2*A*Th/2*np.sinc(kk*Th/(2*T))**2*np.exp(-1j*2*np.pi/T*kk*Th/2)*w
z = np.reshape(z, (t.size,k.size))
z = 1/T*np.sum(z,axis=1)
z = np.real(z)


ax.plot(t,x,color=color1,label='$x(t)$',clip_on=False)
ax.plot(t,y,color=color2,label=r'$y(t)=\frac{1}{2}x\left(t-\frac{T_h}{2}\right)$',clip_on=False)
ax.plot(t,z,color=color3,label=r'$z(t)=x(t)\ast y(t)$',clip_on=False)
ax.legend(loc='upper right')



ax.set_xlim(-5,5)
ax.set_xticks(np.arange(-5,6,1))

ax.set_xticks([-2*T,-T,-Th/2,0,Th/2,T,2*T])
ax.set_xticklabels(labels=['$-2T_0$','$-T_0$',r'$-\faktor{T_h}{2}$','$0$',r'$+\faktor{T_h}{2}$','$+T_0$','$+2T_0$'])


ax.set_ylim(0,A)
ax.set_yticks(np.arange(0,A+A/2,A/2))
ax.set_yticklabels(['$0$',r'$\faktor{A}{2}$','$A$'])
ax.set_title(r'$x(t)=\mathrm{tri}\left(2\frac{t}{T_h}\right),\quad\quad x(t)=x(t+T_0),\quad\quad 0<\frac{T_h}{T}\leq 1$',pad=20)
ax.grid(True)


k = np.arange(-5,5+1,1)
X = A*Th/2*np.sinc(k*Th/(2*T))**2
Y = X*np.exp(-2j*np.pi/T*k*Th/2)/2
Z = X*Y


ax1 = fig.add_subplot(334)
ax2 = fig.add_subplot(335)
ax3 = fig.add_subplot(336)
ax4 = fig.add_subplot(337)
ax5 = fig.add_subplot(338)
ax6 = fig.add_subplot(339)

ax1.set_xlabel('$\mu$')
ax2.set_xlabel('$\mu$')
ax3.set_xlabel('$\mu$')
ax4.set_xlabel('$\mu$')
ax5.set_xlabel('$\mu$')
ax6.set_xlabel('$\mu$')

ax1.plot(k,np.abs(X),marker='o',ls='',mec=color1,mfc=color1,
         label=r'$\left|\tilde{X}\left[\mu\right]\right|$',
         clip_on=False)
ax2.plot(k,np.abs(Y),marker='o',ls='',mec=color1,mfc=color2,
         label=r'$\left|\tilde{Y}\left[\mu\right]\right|$',
         clip_on=False)
ax3.plot(k,np.abs(Z),marker='o',ls='',mec=color1,mfc=color3,
         label=r'$\left|\tilde{Z}\left[\mu\right]\right|$',
         clip_on=False)

ax4.plot(k,np.angle(X),marker='o',ls='',mec=color1,mfc=color1,
         label=r'$\varphi\left(\tilde{X}\left[\mu\right]\right)$',
         clip_on=False)
ax5.plot(k,np.angle(Y),marker='o',ls='',mec=color1,mfc=color2,
         label=r'$\varphi\left(\tilde{Y}\left[\mu\right]\right)$',
         clip_on=False)
ax6.plot(k,np.angle(Z),marker='o',ls='',mec=color1,mfc=color3,
         label=r'$\varphi\left(\tilde{Z}\left[\mu\right]\right)$',
         clip_on=False)

k = np.arange(-5,5+1/1000,1/1000)
X = A*Th/2*np.sinc(k*Th/(2*T))**2
Y = X*np.exp(-2j*np.pi/T*k*Th/2)/2
Z = X*Y

ax1.plot(k,np.abs(X),ls=':',color=color1,clip_on=False)
ax2.plot(k,np.abs(Y),ls=':',color=color2,clip_on=False)
ax3.plot(k,np.abs(Z),ls=':',color=color3,clip_on=False)

ax4.plot(k,np.angle(X),ls=':',color=color1,clip_on=False)
ax5.plot(k,np.angle(Y),ls=':',color=color2,clip_on=False)
ax6.plot(k,np.angle(Z),ls=':',color=color3,clip_on=False)

xticks = np.arange(-5,5+1,1)
yticks = np.arange(-np.pi,np.pi+np.pi/2,np.pi/2)
labels = ['$-\pi$',r'$-\faktor{\pi}{2}$','$0$',r'$+\faktor{\pi}{2}$',r'$+\pi$']

ax1.set_xlim(-5,5)
ax2.set_xlim(-5,5)
ax3.set_xlim(-5,5)
ax4.set_xlim(-5,5)
ax5.set_xlim(-5,5)
ax6.set_xlim(-5,5)

ax1.set_xticks(xticks)
ax2.set_xticks(xticks)
ax3.set_xticks(xticks)
ax4.set_xticks(xticks)
ax5.set_xticks(xticks)
ax6.set_xticks(xticks)

ax1.set_ylim(0,np.amax(np.abs(X)))
ax2.set_ylim(0,np.amax(np.abs(Y)))
ax3.set_ylim(0,np.amax(np.abs(Z)))

ax1.set_yticks(np.arange(0,A*Th/2+A*Th/4,A*Th/4))
ax1.set_yticklabels(['0',r'$\faktor{A T_h}{4}$',
                     r'$\faktor{A T_h}{2}$'])

ax2.set_yticks(np.arange(0,A*Th/4+A*Th/8,A*Th/8))
ax2.set_yticklabels(['0',r'$\faktor{A T_h}{8}$',
                     r'$\faktor{A T_h}{4}$'])
ax3.set_yticks(np.arange(0,A**2*Th**2/8+A**2*Th**2/16,A**2*Th**2/16))
ax3.set_yticklabels(['0',r'$\faktor{A^2 T_h^2}{16}$',
                     r'$\faktor{A^2 T_h^2}{8}$'])

ax4.set_ylim(-np.pi,+np.pi)
ax5.set_ylim(-np.pi,+np.pi)
ax6.set_ylim(-np.pi,+np.pi)

ax4.set_yticks(yticks)
ax5.set_yticks(yticks)
ax6.set_yticks(yticks)

ax4.set_yticklabels(labels)
ax5.set_yticklabels(labels)
ax6.set_yticklabels(labels)


ax1.legend(fontsize=11,loc='center right')
ax2.legend(fontsize=11,loc='center right')
ax3.legend(fontsize=11,loc='center right')
ax4.legend(fontsize=11,loc='upper right')
ax5.legend(fontsize=11,loc='upper right')
ax6.legend(fontsize=11,loc='upper right')

ax1.grid(True)
ax2.grid(True)
ax3.grid(True)
ax4.grid(True)
ax5.grid(True)
ax6.grid(True)

fig.tight_layout()
fig.savefig('fourier_series_tri.pdf')

In [ ]:
T = 2.5
Th = T/2
A = 1.5
color1 = 'black'
color2 = 'C0'
color3 = 'C2'
color4 = 'C3'
color5 = 'C4'

fig = plt.figure(figsize=(9,7))

ax = fig.add_subplot(311)

# time domain
t = np.arange(-5,5+1/1000,1/1000)
x = rect_impulse(t,Th)*A
y = rect_impulse(t-Th/2,Th)*A/2
z = np.zeros_like(t)
i = np.where((-Th/2<=t) & (t < Th/2))
z[i] = A**2/2*(t[i]+Th/2)
i = np.where((Th/2 <= t) & (t < 3*Th/2))
z[i] = A**2/2*(-t[i]+3*Th/2)

ax.plot(t,x,color=color1,label='$x(t)$',clip_on=False)
ax.plot(t,y,color=color2,label=r'$y(t)=\frac{1}{2}x\left(t-\frac{T_h}{2}\right)$',clip_on=False)
ax.plot(t,z,color=color3,label=r'$z(t)=x(t)\ast y(t)$',clip_on=False)
ax.legend(loc='upper right')


# axis settings
ax.set_xlim(-5,5)
ax.set_xticks(np.arange(-5,6,1))

ax.set_xticks([-2*T,-T,-Th/2,0,Th/2,T,2*T])
ax.set_xticklabels(labels=['$-2T$','$-T$',r'$-\faktor{T_h}{2}$','$0$',r'$+\faktor{T_h}{2}$','$+T$','$+2T$'])


ax.set_ylim(0,A)
ax.set_yticks(np.arange(0,A+A/2,A/2))
ax.set_yticklabels(['$0$',r'$\faktor{A}{2}$','$A$'])
ax.set_xlabel('$t$')
ax.set_title(r'$x(t)=\mathrm{rect}\left(\frac{t}{T_h}\right),\quad\quad 0<\frac{T_h}{T}\leq 1$',pad=20)
ax.grid(True)

# frequency domain

omega = np.arange(-5*2*np.pi/T,5*2*np.pi/T+1/1000,1/1000)
X = np.sinc(omega/2*Th/np.pi)*Th*A
Y = np.sinc(omega/2*Th/np.pi)*Th*A/2*np.exp(-1j*omega*Th/2)
Z = X*Y

ax1 = fig.add_subplot(334)
ax2 = fig.add_subplot(335)
ax3 = fig.add_subplot(336)
ax4 = fig.add_subplot(337)
ax5 = fig.add_subplot(338)
ax6 = fig.add_subplot(339)


# swap pi with -pi for omega > 0 (point reflection)
X_ph = np.angle(X)
i = np.where(omega > 0,1,0)
j = np.where(X_ph==np.pi,1,0)
i = np.logical_and(i,j)
i, = np.where(i == True)
X_ph[i] = -np.pi

# plot FT magnitude
ax1.plot(omega,np.abs(X),color=color1,
         label=r'$\left|{X}\left(\omega\right)\right|$',
         clip_on=False)
ax2.plot(omega,np.abs(Y),color=color2,
         label=r'$\left|{Y}\left(\omega\right)\right|$',
         clip_on=False)
ax3.plot(omega,np.abs(Z),color=color3,
         label=r'$\left|{Z}\left(\omega\right)\right|$',
         clip_on=False)
# plot FT phase
ax4.plot(omega,X_ph,color=color1,
         label=r'$\varphi\left({X}\left(\omega\right)\right)$',
         clip_on=False)
ax5.plot(omega,np.angle(Y),color=color2,
         label=r'$\varphi\left({Y}\left(\omega\right)\right)$',
         clip_on=False)
ax6.plot(omega,np.angle(Z),color=color3,
         label=r'$\varphi\left(\tilde{Z}\left(\omega\right)\right)$',
         clip_on=False)


# axis settings for frequency domain
xticks = np.arange(-5*2*np.pi/T,+5*2*np.pi/T+2*np.pi/T,2*np.pi/T)
xlabels = ['$-5\omega_0$','','','$-2\omega_0$','','$0$',
           '','$2\omega_0$','','','$5\omega_0$']

yticks = np.arange(-np.pi,np.pi+np.pi/2,np.pi/2)
ylabels = ['$-\pi$',r'$-\faktor{\pi}{2}$','$0$',r'$+\faktor{\pi}{2}$',r'$+\pi$']



ax1.set_xlim(-5*2*np.pi/T,5*2*np.pi/T)
ax2.set_xlim(-5*2*np.pi/T,5*2*np.pi/T)
ax3.set_xlim(-5*2*np.pi/T,5*2*np.pi/T)
ax4.set_xlim(-5*2*np.pi/T,5*2*np.pi/T)
ax5.set_xlim(-5*2*np.pi/T,5*2*np.pi/T)
ax6.set_xlim(-5*2*np.pi/T,5*2*np.pi/T)

ax1.set_xticks(xticks)
ax2.set_xticks(xticks)
ax3.set_xticks(xticks)
ax4.set_xticks(xticks)
ax5.set_xticks(xticks)
ax6.set_xticks(xticks)

ax1.set_xticklabels(xlabels)
ax2.set_xticklabels(xlabels)
ax3.set_xticklabels(xlabels)
ax4.set_xticklabels(xlabels)
ax5.set_xticklabels(xlabels)
ax6.set_xticklabels(xlabels)

ax1.set_xlabel('$\omega$')
ax2.set_xlabel('$\omega$')
ax3.set_xlabel('$\omega$')
ax4.set_xlabel('$\omega$')
ax5.set_xlabel('$\omega$')
ax6.set_xlabel('$\omega$')

ax1.set_ylim(0,np.amax(np.abs(X)))
ax2.set_ylim(0,np.amax(np.abs(Y)))
ax3.set_ylim(0,np.amax(np.abs(Z)))

ax1.set_yticks(np.arange(0,A*Th+A*Th/2,A*Th/2))
ax1.set_yticklabels(['0',r'$\faktor{A T_h}{2}$',
                     r'$A T_h$'])

ax2.set_yticks(np.arange(0,A*Th/2+A*Th/4,A*Th/4))
ax2.set_yticklabels(['0',r'$\faktor{A T_h}{4}$',
                     r'$\faktor{A T_h}{2}$'])
ax3.set_yticks(np.arange(0,A**2*Th**2/2+A**2*Th**2/4,A**2*Th**2/4))
ax3.set_yticklabels(['0',r'$\faktor{A^2 T_h^2}{4}$',
                     r'$\faktor{A^2 T_h^2}{2}$'])

ax4.set_ylim(-np.pi,+np.pi)
ax5.set_ylim(-np.pi,+np.pi)
ax6.set_ylim(-np.pi,+np.pi)

ax4.set_yticks(yticks)
ax5.set_yticks(yticks)
ax6.set_yticks(yticks)

ax4.set_yticklabels(ylabels)
ax5.set_yticklabels(ylabels)
ax6.set_yticklabels(ylabels)


ax1.legend(fontsize=11,loc='center right')
ax2.legend(fontsize=11,loc='center right')
ax3.legend(fontsize=11,loc='center right')
ax4.legend(fontsize=11,loc='upper right')
ax5.legend(fontsize=11,loc='upper right')
ax6.legend(fontsize=11,loc='upper right')

ax1.grid(True)
ax2.grid(True)
ax3.grid(True)
ax4.grid(True)
ax5.grid(True)
ax6.grid(True)

fig.tight_layout()
fig.savefig('continuous_fouriertrafo_rect.pdf')

In [ ]:
T = 2.5
Th = T/2
A = 1.5
color1 = 'black'
color2 = 'C0'
color3 = 'C2'
color4 = 'C3'
color5 = 'C4'

fig = plt.figure(figsize=(9,7))

ax = fig.add_subplot(311)


# time domain
t = np.arange(-5,5+1/1000,1/1000)
x = tri_impulse(t,Th)*A
y = tri_impulse(t-Th/2,Th)*A/2
z = np.convolve(x,y)/1000
t2 = np.arange(-5*2,2*(5+1/1000),1/1000)
i = np.where((t2 >= -5) & (t2 <= 5))
z = z[i]
t2 = t2[i]

# axis settings
ax.set_xlabel('$t$')
ax.plot(t,x,color=color1,label='$x(t)$',clip_on=False)
ax.plot(t,y,color=color2,label=r'$y(t)=\frac{1}{2}x\left(t-\frac{T_h}{2}\right)$',clip_on=False)
ax.plot(t2,z,color=color3,label=r'$z(t)=x(t)\ast y(t)$',clip_on=False)
ax.legend(loc='upper right')



ax.set_xlim(-5,5)
ax.set_xticks(np.arange(-5,6,1))

ax.set_xticks([-2*T,-T,-Th/2,0,Th/2,T,2*T])
ax.set_xticklabels(labels=['$-2T$','$-T$',r'$-\faktor{T_h}{2}$','$0$',r'$+\faktor{T_h}{2}$','$+T$','$+2T$'])


ax.set_ylim(0,A)
ax.set_yticks(np.arange(0,A+A/2,A/2))
ax.set_yticklabels(['$0$',r'$\faktor{A}{2}$','$A$'])
ax.set_title(r'$x(t)=\mathrm{tri}\left(2\frac{t}{T_h}\right),\quad\quad 0<\frac{T_h}{T}\leq 1$',pad=20)
ax.grid(True)

# frequency domain
omega = np.arange(-5*2*np.pi/T,5*2*np.pi/T+1/1000,1/1000)
X = A*Th/2*np.sinc(omega/4*Th/np.pi)**2
Y = A*Th/4*np.sinc(omega/4*Th/np.pi)**2*np.exp(-1j*omega*Th/2)
Z = X*Y

ax1 = fig.add_subplot(334)
ax2 = fig.add_subplot(335)
ax3 = fig.add_subplot(336)
ax4 = fig.add_subplot(337)
ax5 = fig.add_subplot(338)
ax6 = fig.add_subplot(339)


# plot FT magnitude
ax1.plot(omega,np.abs(X),color=color1,
         label=r'$\left|{X}\left(\omega\right)\right|$',
         clip_on=False)
ax2.plot(omega,np.abs(Y),color=color2,
         label=r'$\left|{Y}\left(\omega\right)\right|$',
         clip_on=False)
ax3.plot(omega,np.abs(Z),color=color3,
         label=r'$\left|{Z}\left(\omega\right)\right|$',
         clip_on=False)
# plot FT phase

ax4.plot(omega,np.angle(X),color=color1,
         label=r'$\varphi\left({X}\left(\omega\right)\right)$',
         clip_on=False)
ax5.plot(omega,np.angle(Y),color=color2,
         label=r'$\varphi\left({Y}\left(\omega\right)\right)$',
         clip_on=False)
ax6.plot(omega,np.angle(Z),color=color3,
         label=r'$\varphi\left({Z}\left(\omega\right)\right)$',
         clip_on=False)

# axis settings

xticks = np.arange(-5*2*np.pi/T,+5*2*np.pi/T+2*np.pi/T,2*np.pi/T)
xlabels = ['$-5\omega_0$','','','$-2\omega_0$','','$0$',
           '','$2\omega_0$','','','$5\omega_0$']

yticks = np.arange(-np.pi,np.pi+np.pi/2,np.pi/2)
ylabels = ['$-\pi$',r'$-\faktor{\pi}{2}$','$0$',r'$+\faktor{\pi}{2}$',r'$+\pi$']



ax1.set_xlim(-5*2*np.pi/T,5*2*np.pi/T)
ax2.set_xlim(-5*2*np.pi/T,5*2*np.pi/T)
ax3.set_xlim(-5*2*np.pi/T,5*2*np.pi/T)
ax4.set_xlim(-5*2*np.pi/T,5*2*np.pi/T)
ax5.set_xlim(-5*2*np.pi/T,5*2*np.pi/T)
ax6.set_xlim(-5*2*np.pi/T,5*2*np.pi/T)

ax1.set_xticks(xticks)
ax2.set_xticks(xticks)
ax3.set_xticks(xticks)
ax4.set_xticks(xticks)
ax5.set_xticks(xticks)
ax6.set_xticks(xticks)

ax1.set_xticklabels(xlabels)
ax2.set_xticklabels(xlabels)
ax3.set_xticklabels(xlabels)
ax4.set_xticklabels(xlabels)
ax5.set_xticklabels(xlabels)
ax6.set_xticklabels(xlabels)

ax1.set_xlabel('$\omega$')
ax2.set_xlabel('$\omega$')
ax3.set_xlabel('$\omega$')
ax4.set_xlabel('$\omega$')
ax5.set_xlabel('$\omega$')
ax6.set_xlabel('$\omega$')

ax1.set_ylim(0,np.amax(np.abs(X)))
ax2.set_ylim(0,np.amax(np.abs(Y)))
ax3.set_ylim(0,np.amax(np.abs(Z)))

ax1.set_yticks(np.arange(0,A*Th/2+A*Th/4,A*Th/4))
ax1.set_yticklabels(['0',r'$\frac{A T_h}{4}$',
                     r'$\frac{A T_h}{2}$'])

ax2.set_yticks(np.arange(0,A*Th/4+A*Th/8,A*Th/8))
ax2.set_yticklabels(['0',r'$\faktor{A T_h}{8}$',
                     r'$\faktor{A T_h}{4}$'])
ax3.set_yticks(np.arange(0,A**2*Th**2/8+A**2*Th**2/16,A**2*Th**2/16))
ax3.set_yticklabels(['0',r'$\faktor{A^2 T_h^2}{16}$',
                     r'$\faktor{A^2 T_h^2}{8}$'])

ax4.set_ylim(-np.pi,+np.pi)
ax5.set_ylim(-np.pi,+np.pi)
ax6.set_ylim(-np.pi,+np.pi)

ax4.set_yticks(yticks)
ax5.set_yticks(yticks)
ax6.set_yticks(yticks)

ax4.set_yticklabels(ylabels)
ax5.set_yticklabels(ylabels)
ax6.set_yticklabels(ylabels)


ax1.legend(fontsize=11,loc='center right')
ax2.legend(fontsize=11,loc='center right')
ax3.legend(fontsize=11,loc='center right')
ax4.legend(fontsize=11,loc='upper right')
ax5.legend(fontsize=11,loc='upper right')
ax6.legend(fontsize=11,loc='upper right')

ax1.grid(True)
ax2.grid(True)
ax3.grid(True)
ax4.grid(True)
ax5.grid(True)
ax6.grid(True)


fig.tight_layout()
fig.savefig('continuous_fouriertrafo_tri.pdf')

&#7838;

In [ ]:
T = 2.5
Th = T/2
A = 1.5
color1 = 'black'
color2 = 'C0'
color3 = 'C2'
color4 = 'C3'
color5 = 'C4'

fig = plt.figure(figsize=(9,7))

ax = fig.add_subplot(311)

# time domain
k = np.arange(-Th/2,Th+Th/4,Th/4)
x = rect_impulse(k,Th)*A
y = rect_impulse(k-Th/2,Th)*A/2
z = np.convolve(x,y)
k2 = np.arange(-Th,2*Th+Th/4,Th/4)

ax.plot(k,x,label='$x[k]$',
        mfc=color1,mec=color1,ls='',marker='o',clip_on=False)
ax.plot(k,y,label=r'$y[k]=\frac{1}{2} x[k-2]$',
        mfc=color2,mec=color2,ls='',marker='o',clip_on=False)
ax.plot(k2,z,label=r'$z[k]=x[k]\ast y[k]$',color=color3,
        mfc=color3,mec=color3,ls=':',marker='o',clip_on=False)
ax.legend(loc='upper right')

#gestrichelte Linien zwischen den Samples

k = np.arange(-2*T,2*T+1/1000,1/1000)
x = rect_impulse(k,Th)*A
y = rect_impulse(k-Th/2,Th)*A/2

ax.plot(k,x,ls=':',color=color1)
ax.plot(k,y,ls=':',color=color2)


#Abwechselnde Farben der Samples
k = np.arange(7*Th/4,2*T+0.01,3*Th/4)
k = np.append(k,np.arange(-2*T+Th/4,-3*Th/4+Th/4,3*Th/4))
plt.plot(k,np.zeros_like(k),ls='',marker='o',mec=color1,mfc=color1,clip_on=False)

k = np.arange(9*Th/4,2*T,3*Th/4)
k = np.append(k,np.arange(-2*T+Th/2,-3*Th/4+Th/4,3*Th/4))
plt.plot(k,np.zeros_like(k),ls='',marker='o',mec=color3,mfc=color3,clip_on=False)

k = np.arange(8*Th/4,2*T,3*Th/4)
k = np.append(k,np.arange(-2*T,-3*Th/4+Th/4,3*Th/4))
plt.plot(k,np.zeros_like(k),ls='',marker='o',mec=color2,mfc=color2,clip_on=False)
#axis settings
ax.set_xlabel('$k$')
ax.set_xlim(-5,5)
ax.set_xticks(np.arange(-5,6,1))

ax.set_xticks(np.arange(-2*T,2*T+Th/4,Th/2))
labels = []
for i in np.arange(-2*T,2*T+Th/4,Th/2):
    labels += ['$'+str(int(4*i/Th))+'$']
ax.set_xticklabels(labels)

ax.set_ylim(0,5*A**2/2)
ax.set_yticks([0,5*A**2/2])
ax.set_yticklabels(['$0$',r'$\faktor{5A^2}{2}$'])
ax.set_title(r'x[k]=$\mathrm{rect}_5[k-2]$',pad=20)
ax.grid(True)

#frequency domain
Omega = np.arange(-2*np.pi,2*np.pi+1/10000,1/10000)
X=A*5*diric(Omega,5)
Y=X/2*np.exp(-2j*Omega)
Z=X*Y

X_ph = np.angle(X)
# swap pi to - pi for 0 <= Omega mod 2pi <= pi (point reflection)
i = np.where(np.mod(Omega,2*np.pi)<=np.pi,1,0)
j = np.where(X_ph==np.pi,1,0)
i = np.logical_and(i,j)
i, = np.where(i == True)
X_ph[i] = -np.pi

ax1 = fig.add_subplot(334)
ax2 = fig.add_subplot(335)
ax3 = fig.add_subplot(336)
ax4 = fig.add_subplot(337)
ax5 = fig.add_subplot(338)
ax6 = fig.add_subplot(339)

ax1.set_xlabel('$\Omega$')
ax2.set_xlabel('$\Omega$')
ax3.set_xlabel('$\Omega$')
ax4.set_xlabel('$\Omega$')
ax5.set_xlabel('$\Omega$')
ax6.set_xlabel('$\Omega$')

# plot DTFT magnitude
ax1.plot(Omega,np.abs(X),color=color1,
         label=r'$\left|{X}\left(\Omega\right)\right|$',
         clip_on=False)
ax2.plot(Omega,np.abs(Y),color=color2,
         label=r'$\left|{Y}\left(\Omega\right)\right|$',
         clip_on=False)
ax3.plot(Omega,np.abs(Z),color=color3,
         label=r'$\left|{Z}\left(\Omega\right)\right|$',
         clip_on=False)
# plot DTFT phase
ax4.plot(Omega,X_ph,color=color1,
         label=r'$\varphi\left({X}\left(\Omega\right)\right)$',
         clip_on=False)
ax5.plot(Omega,np.angle(Y),color=color2,
         label=r'$\varphi\left({Y}\left(\Omega\right)\right)$',
         clip_on=False)
ax6.plot(Omega,np.angle(Z),color=color3,
         label=r'$\varphi\left({Z}\left(\Omega\right)\right)$',
         clip_on=False)


# axis settings
xticks = np.arange(-2*np.pi,2*np.pi+np.pi/2,np.pi)
xlabels = ['$-2\pi$','$-\pi$','$0$','$+\pi$','$+2\pi$']

yticks = np.arange(-np.pi,np.pi+np.pi/2,np.pi/2)
ylabels = ['$-\pi$',r'$-\faktor{\pi}{2}$','$0$',r'$+\faktor{\pi}{2}$',r'$+\pi$']


ax1.set_xlim(-np.pi,+np.pi)
ax2.set_xlim(-np.pi,+np.pi)
ax3.set_xlim(-np.pi,+np.pi)
ax4.set_xlim(-np.pi,+np.pi)
ax5.set_xlim(-np.pi,+np.pi)
ax6.set_xlim(-np.pi,+np.pi)

ax1.set_xticks(xticks)
ax2.set_xticks(xticks)
ax3.set_xticks(xticks)
ax4.set_xticks(xticks)
ax5.set_xticks(xticks)
ax6.set_xticks(xticks)

ax1.set_xticklabels(xlabels)
ax2.set_xticklabels(xlabels)
ax3.set_xticklabels(xlabels)
ax4.set_xticklabels(xlabels)
ax5.set_xticklabels(xlabels)
ax6.set_xticklabels(xlabels)


ax1.set_ylim(0,np.amax(np.abs(X)))
ax2.set_ylim(0,np.amax(np.abs(Y)))
ax3.set_ylim(0,np.amax(np.abs(Z)))

ax1.set_yticks(np.arange(0,5*A+A,A))
ax1.set_yticklabels(['$0$','$A$','$2A$','$3A$','$4A$','$5A$'])

ax2.set_yticks(np.arange(0,5*A/2+A/2,A/2))
ax2.set_yticklabels(['0',r'$\faktor{A}{2}$','$A$',r'$\faktor{3A}{2}$',r'$2A$',r'$\faktor{5A}{2}$'])

ax3.set_yticks(np.arange(0,25*A**2/2+5*A**2/2,5*A**2/2))
ax3.set_yticklabels(['$0$',r'$\faktor{5A^2}{2}$','$5A^2$',r'$\faktor{15A^2}{2}$','$10A^2$',r'$\faktor{25A^2}{2}$'])

ax4.set_ylim(-np.pi,+np.pi)
ax5.set_ylim(-np.pi,+np.pi)
ax6.set_ylim(-np.pi,+np.pi)

ax4.set_yticks(yticks)
ax5.set_yticks(yticks)
ax6.set_yticks(yticks)

ax4.set_yticklabels(ylabels)
ax5.set_yticklabels(ylabels)
ax6.set_yticklabels(ylabels)


ax1.legend(fontsize=11,loc='center right')
ax2.legend(fontsize=11,loc='center right')
ax3.legend(fontsize=11,loc='center right')
ax4.legend(fontsize=11,loc='upper right')
ax5.legend(fontsize=11,loc='upper right')
ax6.legend(fontsize=11,loc='upper right')

ax1.grid(True)
ax2.grid(True)
ax3.grid(True)
ax4.grid(True)
ax5.grid(True)
ax6.grid(True)


fig.tight_layout()
fig.savefig('DTFT_rect.pdf')

In [ ]:
T = 2.5
Th = T/2
A = 1.5
color1 = 'black'
color2 = 'C0'
color3 = 'C2'
color4 = 'C3'
color5 = 'C4'

fig = plt.figure(figsize=(9,7))

ax = fig.add_subplot(311)
# time domain
k = np.arange(-2*T,2*T+0.0001,Th/6)
x = tri_impulse(k,Th)*A
y = tri_impulse(k-Th/2,Th)*A/2
z = np.convolve(x,y)
k2 = np.arange(-4*T,4*T+0.000001,Th/6)
z = z[np.where(np.abs(k2)<=2*T+0.00000001)]


ax.plot(k,x,label='$x[k]$',color=color1,
        mfc=color1,mec=color1,ls=':',marker='o',clip_on=False)
ax.plot(k,y,label=r'$y[k]=\frac{1}{2} x[k-3]$',color=color2,
        mfc=color2,mec=color2,ls=':',marker='o',clip_on=False)
ax.plot(k,z,label=r'$z[k]=x[k]\ast y[k]$',color=color3,
        mfc=color3,mec=color3,ls=':',marker='o',clip_on=False)
ax.legend(loc='upper right')



#Abwechselnde Farben der Samples
k = np.append(np.arange(7,24+0.0000001,3),np.arange(-1,-24-0.0000001,-3))
plt.plot(k*Th/6,np.zeros_like(k),ls='',marker='o',mec=color1,mfc=color1,clip_on=False)

k = np.append(np.arange(6,24+0.0000001,3),np.arange(0,-24-0.00001,-3))
plt.plot(k*Th/6,np.zeros_like(k),ls='',marker='o',mec=color2,mfc=color2,clip_on=False)

k = np.append(np.arange(8,24+0.0000001,3),np.arange(-2,-24-0.0000001,-3))
plt.plot(k*Th/6,np.zeros_like(k),ls='',marker='o',mec=color3,mfc=color3,clip_on=False)
# axis settings
ax.set_xlim(-5,5)
ax.set_xticks(np.arange(-5,6,1))

ax.set_xticks(np.arange(-24,24+0.0000001,3)*Th/6)
labels = []
for i in np.arange(-24,24+0.0000001,3,dtype=int):
    labels += ['$'+str(i)+'$']
ax.set_xticklabels(labels)
ax.set_xlabel('$k$')
ax.set_ylim(0,19/18*A**2)
ax.set_yticks([0,19/18*A**2])
ax.set_yticklabels(['$0$',r'$\faktor{19A^2}{18}$'])
ax.set_title(r'$x[k]=\mathrm{tri}_3[k]$',pad=20)
ax.grid(True)

# frequency domain
Omega = np.arange(-2*np.pi,2*np.pi+1/10000,1/10000)
X = A*diric(Omega,3)**2*3
Y = X/2*np.exp(-3j*Omega)
Z = X*Y

ax1 = fig.add_subplot(334)
ax2 = fig.add_subplot(335)
ax3 = fig.add_subplot(336)
ax4 = fig.add_subplot(337)
ax5 = fig.add_subplot(338)
ax6 = fig.add_subplot(339)

ax1.set_xlabel('$\Omega$')
ax2.set_xlabel('$\Omega$')
ax3.set_xlabel('$\Omega$')
ax4.set_xlabel('$\Omega$')
ax5.set_xlabel('$\Omega$')
ax6.set_xlabel('$\Omega$')

# plot DTFT magnitude
ax1.plot(Omega,np.abs(X),color=color1,
         label=r'$\left|{X}\left(\Omega\right)\right|$',
         clip_on=False)
ax2.plot(Omega,np.abs(Y),color=color2,
         label=r'$\left|{Y}\left(\Omega\right)\right|$',
         clip_on=False)
ax3.plot(Omega,np.abs(Z),color=color3,
         label=r'$\left|{Z}\left(\Omega\right)\right|$',
         clip_on=False)
# plot DTFT phase
ax4.plot(Omega,np.angle(X),color=color1,
         label=r'$\varphi\left({X}\left(\Omega\right)\right)$',
         clip_on=False)
ax5.plot(Omega,np.angle(Y),color=color2,
         label=r'$\varphi\left({Y}\left(\Omega\right)\right)$',
         clip_on=False)
ax6.plot(Omega,np.angle(Z),color=color3,
         label=r'$\varphi\left({Z}\left(\Omega\right)\right)$',
         clip_on=False)


# axis settings
xticks = np.arange(-2*np.pi,2*np.pi+np.pi/2,np.pi)
xlabels = ['$-2\pi$','$-\pi$','$0$','$+\pi$','$+2\pi$']

yticks = np.arange(-np.pi,np.pi+np.pi/2,np.pi/2)
ylabels = ['$-\pi$',r'$-\faktor{\pi}{2}$','$0$',r'$+\faktor{\pi}{2}$',r'$+\pi$']


ax1.set_xlim(-np.pi,+np.pi)
ax2.set_xlim(-np.pi,+np.pi)
ax3.set_xlim(-np.pi,+np.pi)
ax4.set_xlim(-np.pi,+np.pi)
ax5.set_xlim(-np.pi,+np.pi)
ax6.set_xlim(-np.pi,+np.pi)

ax1.set_xticks(xticks)
ax2.set_xticks(xticks)
ax3.set_xticks(xticks)
ax4.set_xticks(xticks)
ax5.set_xticks(xticks)
ax6.set_xticks(xticks)

ax1.set_xticklabels(xlabels)
ax2.set_xticklabels(xlabels)
ax3.set_xticklabels(xlabels)
ax4.set_xticklabels(xlabels)
ax5.set_xticklabels(xlabels)
ax6.set_xticklabels(xlabels)


ax1.set_ylim(0,np.amax(np.abs(X)))
ax2.set_ylim(0,np.amax(np.abs(Y)))
ax3.set_ylim(0,np.amax(np.abs(Z)))

ax1.set_yticks(np.arange(0,3*A+A,A))
ax1.set_yticklabels(['$0$','$A$','$2A$','$3A$'])

ax2.set_yticks(np.arange(0,3*A/2+A/2,A/2))
ax2.set_yticklabels(['0',r'$\faktor{A}{2}$','$A$',r'$\faktor{3A}{2}$'])

ax3.set_yticks(np.arange(0,9*A**2/2+3*A**2/2,3*A**2/2))
ax3.set_yticklabels(['$0$',r'$\faktor{3A^2}{2}$','$3A^2$',r'$\faktor{9A^2}{2}$'])

ax4.set_ylim(-np.pi,+np.pi)
ax5.set_ylim(-np.pi,+np.pi)
ax6.set_ylim(-np.pi,+np.pi)

ax4.set_yticks(yticks)
ax5.set_yticks(yticks)
ax6.set_yticks(yticks)

ax4.set_yticklabels(ylabels)
ax5.set_yticklabels(ylabels)
ax6.set_yticklabels(ylabels)


ax1.legend(fontsize=11,loc='center right')
ax2.legend(fontsize=11,loc='center right')
ax3.legend(fontsize=11,loc='center right')
ax4.legend(fontsize=11,loc='upper right')
ax5.legend(fontsize=11,loc='upper right')
ax6.legend(fontsize=11,loc='upper right')

ax1.grid(True)
ax2.grid(True)
ax3.grid(True)
ax4.grid(True)
ax5.grid(True)
ax6.grid(True)


fig.tight_layout()
fig.savefig('DTFT_tri.pdf')

---
header-includes:
   - \usepackage{amssymb}
   - \usepackage{trfsigns}
---
DTFT eines Dreieickes:
Faltung von $\mathrm{rect}_[k]$ mit sich selbst ergibt ein Dreieck der Länge
2N+1$ mit Maximum bei N-1.

DTFT durch Multiplikation der DTFTs des Rechtecksignals:

\begin{gather}
	\mathrm{rect}_N[k]\ast\mathrm{rect}_N[k]\quad\fourier\quad\e^{-\im\Omega(N-1)}\cdot\frac{\sin^2(N\frac{\Omega}{2})}{\sin^2(\frac{\Omega}{2})}
\end{gather}

Mittelpunkt des Dreiecks soll bei 0 liegen, also Verschiebung um $N-1$ nach 
links.

Durch Faltung steigt das Dreieck jeweils um 1 bis zu einem Maximum 
bei $N-1$ von $N$.

\begin{gather}
	N\cdot \mathrm{tri}_{N}[k]=\e^{-\im\Omega(N-1)}\cdot\frac{\sin^2(N\frac{\Omega}{2})}{\sin^2(\frac{\Omega}{2})}\cdot\e^{+j\Omega(N-1)}=\frac{\sin^2(N\frac{\Omega}{2})}{\sin^2(\frac{\Omega}{2})}
\end{gather}

N muss noch auf die andere Seite gebracht werden.

\begin{gather}
	\mathrm{tri}_{N}[k]\quad\fourier\quad\frac{1}{N}\cdot\frac{\sin^2(N\frac{\Omega}{2})}{\sin^2(\frac{\Omega}{2})}
\end{gather}

Dies ist eine Korrespondenz für einen Dreiecksimpuls der Länge $2N+1$ (bzw. $2N-1$, wenn die Nullen an den Rändern nicht mitgezählt werden) mit der
Spitze bei $k=0$.

\begin{gather}
	\mathrm{tri}_N[k]=
	\begin{cases}
		1-\frac{k}{N}, &0 \leq k \leq N-1,\\
		1+\frac{k}{N}, &-N+1\leq k < 0,\\
		0, &\text{sonst}.
	\end{cases}
\end{gather}



\begin{gather}
    \lim\limits_{\Omega\rightarrow0}\frac{1}{N}\cdot\frac{\sin^2(N\frac{\Omega}{2})}{\sin^2(\frac{\Omega}{2})}
    =\lim\limits_{\Omega\rightarrow0}\frac{1}{N}\cdot\frac{2\sin\left(N\frac{\Omega}{2}\right)\cos\left(N\frac{\Omega}{2}\right)\frac{N}{2}}{2\sin\left(\frac{\Omega}{2}\right)\cos\left(\frac{\Omega}{2}\right)\frac{1}{2}}\\
	=\lim\limits_{\Omega\rightarrow0}
	\frac{
		\cos^2\left(N\frac{\Omega}{2}\right)\frac{N}{2}-\sin^2\left(N\frac{\Omega}{2}\right)\frac{N}{2}
	}
	{
		\cos^2\left(\frac{\Omega}{2}\right)\frac{1}{2}-\sin^2\left(\frac{\Omega}{2}\right)\frac{1}{2}
	}
	=N
\end{gather}


In [ ]:
T = 2.5
Th = T/2
A = 1.5
color1 = 'black'
color2 = 'C0'
color3 = 'C2'
color4 = 'C3'
color5 = 'C4'

fig = plt.figure(figsize=(9,7))

ax = fig.add_subplot(311)

# time domain

k = np.arange(0,8,1/1000)
x = A*(square((k+2)/8*2*np.pi,0.5)+1)/2
y = A/2*(square(k/8*2*np.pi,0.5)+1)/2

ax.plot(k,x,ls='--',color=color1,clip_on=False)
ax.plot(k,y,ls='--',color=color2,clip_on=False)

k = np.arange(-16,0,1/1000)
x = A*(square((k+2)/8*2*np.pi,0.5)+1)/2
y = A/2*(square(k/8*2*np.pi,0.5)+1)/2

ax.plot(k,x,ls=':',color=color1,clip_on=False)
ax.plot(k,y,ls=':',color=color2,clip_on=False)

k = np.arange(8,16+1/1000,1/1000)
x = A*(square((k+2)/8*2*np.pi,0.5)+1)/2
y = A/2*(square(k/8*2*np.pi,0.5)+1)/2

ax.plot(k,x,ls=':',color=color1,clip_on=False)
ax.plot(k,y,ls=':',color=color2,clip_on=False)

k = np.arange(0,8,dtype=float)
x = np.zeros_like(k)
y = np.zeros_like(k)

np.put(x,[0,1,2,6,7],A)
#np.put(x,[3,9],A/2)
np.put(y,[0,1,2,3,4],A/2)
#np.put(y,[0,6],A/4)
z = np.real(np.fft.ifft(np.fft.fft(x)*np.fft.fft(y)))


#plot period 2, 3 (first element)
k2 = np.append(np.append(k-16,k-8),k[0])
x2 = np.append(np.append(x,x),x[0])
y2 = np.append(np.append(y,y),y[0])
z2 = np.append(np.append(z,z),z[0])

ax.plot(k2,x2,ls='',color=color1,
        marker='o',mfc='white',mec=color1,clip_on=False)
ax.plot(k2,y2,ls='',color=color2,
        marker='o',mfc='white',mec=color2,clip_on=False)
ax.plot(k2,z2,ls=':',color=color3,
        marker='o',mfc='white',mec=color3,clip_on=False)
#plot period -2 and -1

k2 = np.append(k[k.size-1],np.append(k+8,k[0]+16))
x2 = np.append(x[x.size-1],np.append(x,x[0]))
y2 = np.append(y[y.size-1],np.append(y,y[0]))
z2 = np.append(z[z.size-1],np.append(z,z[0]))

ax.plot(k2,x2,ls='',color=color1,
        marker='o',mfc='white',mec=color1,clip_on=False)
ax.plot(k2,y2,ls='',color=color2,
        marker='o',mfc='white',mec=color2,clip_on=False)
ax.plot(k2,z2,ls=':',color=color3,
        marker='o',mfc='white',mec=color3,clip_on=False)


##########################################

#plot period 1
ax.plot(k,x,label='$x[k]$',color=color1,
        mfc=color1,mec=color1,ls='',marker='o',clip_on=False)
ax.plot(k,y,label=r'$y[k]=\frac{1}{2} x[k-2]$',color=color2,
        mfc=color2,mec=color2,ls='',marker='o',clip_on=False)
ax.plot(k,z,label=r'$z[k]=x[k]\circledast_{8} y[k]$',color=color3,
        mfc=color3,mec=color3,ls='--',marker='o',clip_on=False)
ax.legend(loc='upper right')


##########################################
# axis settings

ax.set_xlabel('$k$')
ax.set_xlim(-16,16)
ax.set_xticks(np.arange(-16,16+2,2))

labels = []
for i in np.arange(-16,16+2,2):
    labels += ['$'+str(i)+'$']
ax.set_xticklabels(labels)

ax.set_ylim(0,5*A**2/2)
ax.set_yticks([0,5*A**2/2])
ax.set_yticklabels(['$0$',r'$\faktor{5A^2}{2}$'])
ax.set_title(r'$x[k]=\mathrm{rect}_5[(k-2) \bmod 8]$')
ax.grid(True)
##########################################
# frequency domain
X = np.fft.fft(x)
Y = np.fft.fft(y)
Z = X*Y

X_ph = np.angle(X)
X_ph[2] = -np.pi
X_ph[3] = -np.pi

ax1 = fig.add_subplot(334)
ax2 = fig.add_subplot(335)
ax3 = fig.add_subplot(336)
ax4 = fig.add_subplot(337)
ax5 = fig.add_subplot(338)
ax6 = fig.add_subplot(339)
##########################################
# plot DTFT magnitude/phase for -2pi <= Omega <= 0
Omega = np.arange(0,2*np.pi+1/1000,1/1000)
X_dtft = A*diric(Omega,5)*5
Y_dtft = X_dtft/2*np.exp(-2j*Omega)
Z_dtft = X_dtft*Y_dtft
X_dtft_ph = np.angle(X_dtft)
# exchange pi to -pi for point reflection (DTFT phase)
i = np.where(Omega <= np.pi,1,0)
j = np.where(X_dtft_ph == np.pi,1,0)
i = np.logical_and(i,j)
i, = np.where(i == True)
X_dtft_ph[i] = -np.pi
# plot DTFT magnitude
ax1.plot(Omega*4/np.pi,np.abs(X_dtft),ls='--',color=color1,clip_on=False)
ax2.plot(Omega*4/np.pi,np.abs(Y_dtft),ls='--',color=color2,clip_on=False)
ax3.plot(Omega*4/np.pi,np.abs(Z_dtft),ls='--',color=color3,clip_on=False)
# plot DTFT phase
ax4.plot(Omega*4/np.pi,X_dtft_ph,ls='--',color=color1,clip_on=False)
ax5.plot(Omega*4/np.pi,np.angle(Y_dtft),ls='--',color=color2,clip_on=False)
ax6.plot(Omega*4/np.pi,np.angle(Z_dtft),ls='--',color=color3,clip_on=False)

##########################################



#plot DTFT magnitude/phase -2pi <= Omega <= 0
Omega = np.arange(-2*np.pi,1/1000,1/1000)

X_dtft = A*diric(Omega,5)*5
Y_dtft = X_dtft/2*np.exp(-2j*Omega)
Z_dtft = X_dtft*Y_dtft
X_dtft_ph = np.angle(X_dtft)
# exchange pi to -pi for point reflection (DTFT phase)
i = np.where(Omega <= -np.pi,1,0)
j = np.where(X_dtft_ph == np.pi,1,0)
i = np.logical_and(i,j)
i, = np.where(i == True)
X_dtft_ph[i] = -np.pi

ax1.plot(Omega*4/np.pi,np.abs(X_dtft),ls=':',color=color1,clip_on=False)
ax2.plot(Omega*4/np.pi,np.abs(Y_dtft),ls=':',color=color2,clip_on=False)
ax3.plot(Omega*4/np.pi,np.abs(Z_dtft),ls=':',color=color3,clip_on=False)

ax4.plot(Omega*4/np.pi,X_dtft_ph,ls=':',color=color1,clip_on=False)
ax5.plot(Omega*4/np.pi,np.angle(Y_dtft),ls=':',color=color2,clip_on=False)
ax6.plot(Omega*4/np.pi,np.angle(Z_dtft),ls=':',color=color3,clip_on=False)


##########################################
#plot DFT period 1
ax1.plot(k,np.abs(X),ls='',marker='o',mfc=color1,mec=color1,
         label=r'$\left|{X}\left[\mu\right]\right|$',
         clip_on=False)
ax2.plot(k,np.abs(Y),ls='',marker='o',mfc=color2,mec=color2,
         label=r'$\left|{Y}\left[\mu\right]\right|$',
         clip_on=False)
ax3.plot(k,np.abs(Z),ls='',marker='o',mfc=color3,mec=color3,
         label=r'$\left|{Z}\left[\mu\right]\right|$',
         clip_on=False)

ax4.plot(k,X_ph,ls='',marker='o',mfc=color1,mec=color1,
         label=r'$\varphi\left({X}\left[\mu\right]\right)$',
         clip_on=False)
ax5.plot(k,np.angle(Y),ls='',marker='o',mfc=color2,mec=color2,
         label=r'$\varphi\left({Y}\left[\mu\right]\right)$',
         clip_on=False)
ax6.plot(k,np.angle(Z),ls='',marker='o',mfc=color3,mec=color3,
         label=r'$\varphi\left({Z}\left[\mu\right]\right)$',
         clip_on=False)
##########################################
# plot DFT magnitude/phase period -1,2 (first element)
mu = np.append(k-8,k[0]+8)
X2 = np.append(X,X[0])
Y2 = np.append(Y,Y[0])
Z2 = np.append(Z,Z[0])
X2_ph = np.append(X_ph,X_ph[0])

ax1.plot(mu,np.abs(X2),ls='',marker='o',mec=color1,mfc='white',clip_on=False)
ax2.plot(mu,np.abs(Y2),ls='',marker='o',mec=color2,mfc='white',clip_on=False)
ax3.plot(mu,np.abs(Z2),ls='',marker='o',mec=color3,mfc='white',clip_on=False)

ax4.plot(mu,X2_ph,ls='',marker='o',mec=color1,mfc='white',clip_on=False)
ax5.plot(mu,np.angle(Y2),ls='',marker='o',mec=color2,mfc='white',clip_on=False)
ax6.plot(mu,np.angle(Z2),ls='',marker='o',mec=color3,mfc='white',clip_on=False)


#####################################################################
#axis settings

xticks = np.arange(-8,8+1,4)
xlabels = ['$-8$','$-4$','$0$','$+4$','$+8$']

yticks = np.arange(-np.pi,np.pi+np.pi/2,np.pi/2)
ylabels = ['$-\pi$',r'$-\faktor{\pi}{2}$','$0$',r'$+\faktor{\pi}{2}$',r'$+\pi$']

ax1.set_xlim(-k.size,k.size)
ax2.set_xlim(-k.size,k.size)
ax3.set_xlim(-k.size,k.size)
ax4.set_xlim(-k.size,k.size)
ax5.set_xlim(-k.size,k.size)
ax6.set_xlim(-k.size,k.size)


ax1.set_xticks(xticks)
ax2.set_xticks(xticks)
ax3.set_xticks(xticks)
ax4.set_xticks(xticks)
ax5.set_xticks(xticks)
ax6.set_xticks(xticks)

ax1.set_xticklabels(xlabels)
ax2.set_xticklabels(xlabels)
ax3.set_xticklabels(xlabels)
ax4.set_xticklabels(xlabels)
ax5.set_xticklabels(xlabels)
ax6.set_xticklabels(xlabels)

ax1.set_xlabel('$\mu$')
ax2.set_xlabel('$\mu$')
ax3.set_xlabel('$\mu$')
ax4.set_xlabel('$\mu$')
ax5.set_xlabel('$\mu$')
ax6.set_xlabel('$\mu$')

ax1.set_ylim(0,np.amax(np.abs(X)))
ax2.set_ylim(0,np.amax(np.abs(Y)))
ax3.set_ylim(0,np.amax(np.abs(Z)))

ax1.set_yticks(np.arange(0,5*A+A,A))
ax1.set_yticklabels(['$0$','$A$','$2A$','$3A$','$4A$','$5A$'])

ax2.set_yticks(np.arange(0,5*A/2+A/2,A/2))
ax2.set_yticklabels(['0',r'$\faktor{A}{2}$','$A$',r'$\faktor{3A}{2}$',r'$2A$',r'$\faktor{5A}{2}$'])

ax3.set_yticks(np.arange(0,25*A**2/2+5*A**2/2,5*A**2/2))
ax3.set_yticklabels(['$0$',r'$\faktor{5A^2}{2}$','$5A^2$',r'$\faktor{15A^2}{2}$','$10A^2$',r'$\faktor{25A^2}{2}$'])

ax4.set_ylim(-np.pi,+np.pi)
ax5.set_ylim(-np.pi,+np.pi)
ax6.set_ylim(-np.pi,+np.pi)

ax4.set_yticks(yticks)
ax5.set_yticks(yticks)
ax6.set_yticks(yticks)

ax4.set_yticklabels(ylabels)
ax5.set_yticklabels(ylabels)
ax6.set_yticklabels(ylabels)


ax1.legend(fontsize=11,loc='center right')
ax2.legend(fontsize=11,loc='center right')
ax3.legend(fontsize=11,loc='center right')
ax4.legend(fontsize=11,loc='upper right')
ax5.legend(fontsize=11,loc='upper right')
ax6.legend(fontsize=11,loc='upper right')

ax1.grid(True)
ax2.grid(True)
ax3.grid(True)
ax4.grid(True)
ax5.grid(True)
ax6.grid(True)


fig.tight_layout()
fig.savefig('DFT_rect.pdf')

In [ ]:
T = 2.5
Th = T/2
A = 1.5
color1 = 'black'
color2 = 'C0'
color3 = 'C2'
color4 = 'C3'
color5 = 'C4'

fig = plt.figure(figsize=(11,9))

ax = fig.add_subplot(311)


# time domain
k = np.arange(0,12,dtype=float)
x = np.zeros_like(k)
y = np.zeros_like(k)

np.put(x,0,A)
np.put(x,1,2*A/3)
np.put(x,2,A/3)
np.put(x,11,2*A/3)
np.put(x,10,A/3)


np.put(y,1,A/6)
np.put(y,2,A/3)
np.put(y,3,A/2)
np.put(y,4,A/3)
np.put(y,5,A/6)

z = np.real(np.fft.ifft(np.fft.fft(x)*np.fft.fft(y)))

#plot dashed line period 1
k2 = np.append(k,k[0]+12)
x2 = np.append(x,x[0])
y2 = np.append(y,y[0])
z2 = np.append(z,z[0])

ax.plot(k2,x2,ls='--',color=color1,
        marker='o',mfc=color1,mec=color1,clip_on=False)
ax.plot(k2,y2,ls='--',color=color3,
        marker='o',mfc=color2,mec=color2,clip_on=False)
ax.plot(k2,z2,ls='--',color=color3,
        marker='o',mfc=color3,mec=color3,clip_on=False)




#plot period 2, 3 (first element)
k2 = np.append(np.append(k-24,k-12),k[0])
x2 = np.append(np.append(x,x),x[0])
y2 = np.append(np.append(y,y),y[0])
z2 = np.append(np.append(z,z),z[0])

ax.plot(k2,x2,ls=':',color=color1,
        marker='o',mfc='white',mec=color1,clip_on=False)
ax.plot(k2,y2,ls=':',color=color2,
        marker='o',mfc='white',mec=color2,clip_on=False)
ax.plot(k2,z2,ls=':',color=color3,
        marker='o',mfc='white',mec=color3,clip_on=False)


#plot period -2 and -1

k2 = np.append(k+12,k[0]+24)
x2 = np.append(x,x[0])
y2 = np.append(y,y[0])
z2 = np.append(z,z[0])

ax.plot(k2,x2,ls=':',color=color1,
        marker='o',mfc='white',mec=color1,clip_on=False)
ax.plot(k2,y2,ls=':',color=color2,
        marker='o',mfc='white',mec=color2,clip_on=False)
ax.plot(k2,z2,ls=':',color=color3,
        marker='o',mfc='white',mec=color3,clip_on=False)


##########################################

#plot period 1
ax.plot(k,x,label='$x[k]$',color=color1,
        mfc=color1,mec=color1,ls='--',marker='o',clip_on=False)
ax.plot(k,y,label=r'$y[k]=\frac{1}{2} x[k-3]$',color=color2,
        mfc=color2,mec=color2,ls='--',marker='o',clip_on=False)
ax.plot(k,z,label=r'$z[k]=x[k]\circledast_{12} y[k]$',color=color3,
        mfc=color3,mec=color3,ls='--',marker='o',clip_on=False)
ax.legend(loc='upper right')


##########################################
# axis settings
ax.set_xlim(-24,24)
ax.set_xticks(np.arange(-24,24+3,3))
ax.set_xlabel('$k$')
labels = []
for i in np.arange(-24,24+3,3):
    labels += ['$'+str(i)+'$']
ax.set_xticklabels(labels)

ax.set_ylim(0,19/18*A**2)
ax.set_yticks([0,A/2,A,19/18*A**2])
ax.set_yticklabels(['$0$',r'$\faktor{A}{2}$','$A$',r'$\faktor{19A^2}{18}$'])
ax.set_yticks([0,19/18*A**2])
ax.set_title(r'$x[k]=\mathrm{tri}_3[k\bmod 12]$')
ax.grid(True)

# frequency domain
X = np.fft.fft(x)
Y = np.fft.fft(y)
Z = X*Y



ax1 = fig.add_subplot(334)
ax2 = fig.add_subplot(335)
ax3 = fig.add_subplot(336)
ax4 = fig.add_subplot(337)
ax5 = fig.add_subplot(338)
ax6 = fig.add_subplot(339)

ax1.set_xlabel('$\mu$')
ax2.set_xlabel('$\mu$')
ax3.set_xlabel('$\mu$')
ax4.set_xlabel('$\mu$')
ax5.set_xlabel('$\mu$')
ax6.set_xlabel('$\mu$')

##########################################


#plot DFT magnitude period 1
ax1.plot(k,np.abs(X),ls='',marker='o',mfc=color1,mec=color1,
         label=r'$\left|{X}\left[\mu\right]\right|$',
         clip_on=False)
ax2.plot(k,np.abs(Y),ls='',marker='o',mfc=color2,mec=color2,
         label=r'$\left|{Y}\left[\mu\right]\right|$',
         clip_on=False)
ax3.plot(k,np.abs(Z),ls='',marker='o',mfc=color3,mec=color3,
         label=r'$\left|{Z}\left[\mu\right]\right|$',
         clip_on=False)


##########################################



#plot DFT phase period 1
ax4.plot(k,np.angle(X),ls='',marker='o',mfc=color1,mec=color1,
         label=r'$\varphi\left({X}\left[\mu\right]\right)$',
         clip_on=False)
ax5.plot(k,np.angle(Y),ls='',marker='o',mfc=color2,mec=color2,
         label=r'$\varphi\left({Y}\left[\mu\right]\right)$',
         clip_on=False)
ax6.plot(k,np.angle(Z),ls='',marker='o',mfc=color3,mec=color3,
         label=r'$\varphi\left({Z}\left[\mu\right]\right)$',
         clip_on=False)

mu = np.append(k-12,k[0]+12)
X2 = np.append(X,X[0])
Y2 = np.append(Y,Y[0])
Z2 = np.append(Z,Z[0])

"""#plot DFT magnitude period -1, 2 (first element)
ax1.plot(mu,np.abs(X2),ls='',marker='o',mec=color1,mfc='white',clip_on=False)
ax2.plot(mu,np.abs(Y2),ls='',marker='o',mec=color2,mfc='white',clip_on=False)
ax3.plot(mu,np.abs(Z2),ls='',marker='o',mec=color3,mfc='white',clip_on=False)

#plot DFT phase period -1, 2 (first element)
ax4.plot(mu,np.angle(X2),ls='',marker='o',mec=color1,mfc='white',clip_on=False)
ax5.plot(mu,np.angle(Y2),ls='',marker='o',mec=color2,mfc='white',clip_on=False)
ax6.plot(mu,np.angle(Z2),ls='',marker='o',mec=color3,mfc='white',clip_on=False)"""

##########################################


#plot DTFT magnitude/phase 0 <= Omega <= 2 pi
Omega = np.arange(0,2*np.pi+1/1000,1/1000)

X_dtft = A*diric(Omega,3)**2*3
Y_dtft = X_dtft/2*np.exp(-3j*Omega)
Z_dtft = X_dtft*Y_dtft
# plot DTFT magnitude
ax1.plot(Omega*6/np.pi,np.abs(X_dtft),ls='--',color=color1,clip_on=False)
ax2.plot(Omega*6/np.pi,np.abs(Y_dtft),ls='--',color=color2,clip_on=False)
ax3.plot(Omega*6/np.pi,np.abs(Z_dtft),ls='--',color=color3,clip_on=False)
# plot DTFT phase
ax4.plot(Omega*6/np.pi,np.angle(X_dtft),ls='--',color=color1,clip_on=False)
ax5.plot(Omega*6/np.pi,np.angle(Y_dtft),ls='--',color=color2,clip_on=False)
ax6.plot(Omega*6/np.pi,np.angle(Z_dtft),ls='--',color=color3,clip_on=False)

##########################################



#plot DTFT magnitude/phase -2pi <= Omega <= 0
Omega = np.arange(-2*np.pi,1/1000,1/1000)

X_dtft = A*diric(Omega,3)**2*3
Y_dtft = X_dtft/2*np.exp(-3j*Omega)
Z_dtft = X_dtft*Y_dtft

ax1.plot(Omega*6/np.pi,np.abs(X_dtft),ls=':',color=color1,clip_on=False)
ax2.plot(Omega*6/np.pi,np.abs(Y_dtft),ls=':',color=color2,clip_on=False)
ax3.plot(Omega*6/np.pi,np.abs(Z_dtft),ls=':',color=color3,clip_on=False)

ax4.plot(Omega*6/np.pi,np.angle(X_dtft),ls=':',color=color1,clip_on=False)
ax5.plot(Omega*6/np.pi,np.angle(Y_dtft),ls=':',color=color2,clip_on=False)
ax6.plot(Omega*6/np.pi,np.angle(Z_dtft),ls=':',color=color3,clip_on=False)


##########################################



#plot DFT magnitude period -1, 2 (first element)
ax1.plot(mu,np.abs(X2),ls='',marker='o',mec=color1,mfc='white',clip_on=False)
ax2.plot(mu,np.abs(Y2),ls='',marker='o',mec=color2,mfc='white',clip_on=False)
ax3.plot(mu,np.abs(Z2),ls='',marker='o',mec=color3,mfc='white',clip_on=False)


##########################################


#plot DFT phase period -1, 2 (first element)
ax4.plot(mu,np.angle(X2),ls='',marker='o',mec=color1,mfc='white',clip_on=False)
ax5.plot(mu,np.angle(Y2),ls='',marker='o',mec=color2,mfc='white',clip_on=False)
ax6.plot(mu,np.angle(Z2),ls='',marker='o',mec=color3,mfc='white',clip_on=False)


##########################################

# axis settings
xticks = np.arange(-12,12+1,4)
xlabels = ['$-12$','$-8$','$-4$','$0$','$+4$','$+8$','$+12$']

yticks = np.arange(-np.pi,np.pi+np.pi/2,np.pi/2)
ylabels = ['$-\pi$',r'$-\faktor{\pi}{2}$','$0$',r'$+\faktor{\pi}{2}$',r'$+\pi$']

ax1.set_xlim(-k.size,k.size)
ax2.set_xlim(-k.size,k.size)
ax3.set_xlim(-k.size,k.size)
ax4.set_xlim(-k.size,k.size)
ax5.set_xlim(-k.size,k.size)
ax6.set_xlim(-k.size,k.size)


ax1.set_xticks(xticks)
ax2.set_xticks(xticks)
ax3.set_xticks(xticks)
ax4.set_xticks(xticks)
ax5.set_xticks(xticks)
ax6.set_xticks(xticks)

ax1.set_xticklabels(xlabels)
ax2.set_xticklabels(xlabels)
ax3.set_xticklabels(xlabels)
ax4.set_xticklabels(xlabels)
ax5.set_xticklabels(xlabels)
ax6.set_xticklabels(xlabels)


ax1.set_ylim(0,np.amax(np.abs(X)))
ax2.set_ylim(0,np.amax(np.abs(Y)))
ax3.set_ylim(0,np.amax(np.abs(Z)))

ax1.set_yticks(np.arange(0,3*A+A,A))
ax1.set_yticklabels(['$0$','$A$','$2A$','$3A$'])

ax2.set_yticks(np.arange(0,3*A/2+A/2,A/2))
ax2.set_yticklabels(['0',r'$\faktor{A}{2}$','$A$',r'$\faktor{3A}{2}$'])

ax3.set_yticks(np.arange(0,9*A**2/2+3*A**2/2,3*A**2/2))
ax3.set_yticklabels(['$0$',r'$\faktor{3A^2}{2}$','$3A^2$',r'$\faktor{9A^2}{2}$'])

ax4.set_ylim(-np.pi,+np.pi)
ax5.set_ylim(-np.pi,+np.pi)
ax6.set_ylim(-np.pi,+np.pi)

ax4.set_yticks(yticks)
ax5.set_yticks(yticks)
ax6.set_yticks(yticks)

ax4.set_yticklabels(ylabels)
ax5.set_yticklabels(ylabels)
ax6.set_yticklabels(ylabels)


ax1.legend(fontsize=11,loc='center right')
ax2.legend(fontsize=11,loc='center right')
ax3.legend(fontsize=11,loc='center right')
ax4.legend(fontsize=11,loc='upper right')
ax5.legend(fontsize=11,loc='upper right')
ax6.legend(fontsize=11,loc='upper right')

ax1.grid(True)
ax2.grid(True)
ax3.grid(True)
ax4.grid(True)
ax5.grid(True)
ax6.grid(True)


fig.tight_layout()
fig.savefig('DFT_tri.pdf')

In [ ]:
sigma_0 = Symbol('\sigma_0',positive=True)
s = Symbol('s')
t = Symbol('t')
A = Symbol('A')
omega = Symbol('\omega')
omega_0 = Symbol('\omega_0',positive=True)
F = A/(s+sigma_0)
f = transforms.inverse_laplace_transform(F,s,t)
display(Math(latex(f)))
F = A*(s-sigma_0)/(s+sigma_0)
F = apart(F,s)
f = transforms.inverse_laplace_transform(F,s,t)
display(Math(latex(f)))
F = (s+sigma_0/4)/(s-(-sigma_0/4+I*omega_0))/(s-(-sigma_0/4-I*omega_0))
F = apart(F,s)
f = transforms.inverse_laplace_transform(F,s,t)
display(Math(latex(f)))

In [ ]:
fig, ((ax1,ax2),(ax3,ax4),(ax5,ax6)) = plt.subplots(ncols=2,nrows=3,gridspec_kw=dict(width_ratios=[1,2.25]),figsize=(11,9))

################################################################################
# zeros, poles, gains

omega_0 = 1
sigma_0 = 1
A=1
s_infty0=-sigma_0+0j*omega_0
s_zero0=sigma_0+0j*omega_0
s_zero1=-sigma_0/4
s_infty1=-sigma_0/4+1j*omega_0
s_infty2=-sigma_0/4-1j*omega_0
s = np.arange(np.real(s_infty0),2+1/1000,1/1000)
s2 = np.arange(np.real(s_infty1),2+1/1000,1/1000)

################################################################################

# s-plane

ax1.fill_between(s,
                 np.zeros_like(s)-2,
                 np.zeros_like(s)+2,
                 color='C2',
                 clip_on=True,
                 label=r'$\mathrm{ROC}\{s\}$',
                 alpha=0.5,
                )
ax1.set_xlabel(r'$\mathrm{Re}\{s\}$')
ax1.set_ylabel(r'$\mathrm{Im}\{s\}$')
ax1.scatter(np.real(s_infty0),np.imag(s_infty0),
            s=75,
            marker='x',
            color='C0'
           )
ax1.set_xlim(-2*sigma_0,2*sigma_0)
ax1.set_ylim(-2*omega_0,2*omega_0)
ax1.set_xticks(np.arange(-2,2+1,1))
ax1.set_yticks(np.arange(-2,2+1,1))
ax1.set_xticklabels([r'$-2\sigma_0$',r'$-\sigma_0$','$0$',r'$+\sigma_0$',r'$+2\sigma_0$'])
ax1.set_yticklabels([r'$-2\omega_0$',r'$-\omega_0$',r'$0$',r'$+\omega_0$',r'$+2\omega_0$'])
ax1.text(1,-1.5,
         s='$g=A$',
         bbox=dict(boxstyle='square',ec='black',fc='white'),
         ha='center')
ax1.grid(True)
ax1.annotate(r'$s_{\infty,0}$',
             (np.real(s_infty0),np.imag(s_infty0)),
             (np.real(s_infty0),np.imag(s_infty0)-0.5),
            size='x-large',
            ha = 'center')
ax1.set_box_aspect(1)
ax1.legend(edgecolor='black')
ax1.set_title(r'$H(s)=A\frac{1}{s+\sigma_0}$',pad=10)

# time-domain

system = signal.lti([],[s_infty0],A)
t = np.arange(0,7*np.log(2)/sigma_0,1/1000)
t, impulse = signal.lti.impulse(system,T=t)
ax2.plot(t,impulse,clip_on=False)
ax2.set_ylim(0,A)
ax2.set_xlim(1,-np.log(2**7)/np.real(s_infty0))
ax2.set_yticks([0,A/8,A/4,A/2,A])
ax2.set_yticklabels(['$0$',r'$\faktor{A}{8}$',r'$\faktor{A}{4}$',r'$\faktor{A}{2}$','$A$'])
ax2.set_xticks(np.arange(0,-np.log(2**8)/np.real(s_infty0),-np.log(2)/np.real(s_infty0)))
xticklabels=['$0$',r'$\frac{\ln(2)}{\sigma_0}$']
for i in range(2,8):
    xticklabels += [r'$\frac{%d\ln(2)}{\sigma_0}$'%(i)]
ax2.set_xticklabels(xticklabels)
ax2.grid(True)
ax2.set_title(r'$h(t)=A\mathrm{e}^{-\sigma_0t}\epsilon(t)$',pad=10)
ax2.set_xlabel('$t$')

################################################################################

ax3.fill_between(s,
                 np.zeros_like(s)-2,
                 np.zeros_like(s)+2,
                 color='C2',
                 clip_on=True,
                 label=r'$\mathrm{ROC}\{s\}$',
                 alpha=0.5,
                )
ax3.set_xlabel(r'$\mathrm{Re}\{s\}$')
ax3.set_ylabel(r'$\mathrm{Im}\{s\}$')
ax3.scatter(np.real(s_infty0),np.imag(s_infty0),
            s=75,
            marker='x',
            color='C0'
           )

ax3.scatter(np.real(s_zero0),np.imag(s_zero0),
            s=75,
            marker='o',
            ec='C0',
            fc='none'
           )
ax3.set_xlim(-2*sigma_0,2*sigma_0)
ax3.set_ylim(-2*omega_0,2*omega_0)
ax3.set_xticks(np.arange(-2,2+1,1))
ax3.set_yticks(np.arange(-2,2+1,1))
ax3.set_xticklabels([r'$-2\sigma_0$',r'$-\sigma_0$','$0$',r'$+\sigma_0$',r'$+2\sigma_0$'])
ax3.set_yticklabels([r'$-2\omega_0$',r'$-\omega_0$',r'$0$',r'$+\omega_0$',r'$+2\omega_0$'])
ax3.text(1,-1.5,
         s='$g=A$',
         bbox=dict(boxstyle='round',ec='black',fc='white'),
         ha='center')
ax3.grid(True)
ax3.annotate(r'$s_{\infty,0}$',
             (np.real(s_infty0),np.imag(s_infty0)),
             (np.real(s_infty0),np.imag(s_infty0)-0.5),
            size='x-large',
            ha = 'center')

ax3.annotate(r'$s_{0,0}$',
             (np.real(s_zero0),np.imag(s_zero0)),
             (np.real(s_zero0),np.imag(s_zero0)-0.5),
            size='x-large',
            ha = 'center')
ax3.set_box_aspect(1)
ax3.legend(edgecolor='black')

ax3.set_title(r'$H(s)=A\frac{s-\sigma_0}{s+\sigma_0}$',pad=10)

# time-domain

system = signal.lti([s_zero0],[s_infty0],A)
t, impulse = signal.lti.impulse(system,T=t)
ax4.plot(t,impulse,clip_on=False)
ax4.annotate(text='',xy=(0,0),xytext=(0,A),
             arrowprops=dict(arrowstyle='<-',
                             connectionstyle='arc3',
                             shrinkA=0,shrinkB=0,
                             ec='C0',fc='C0',
                             lw=1.5,
                             clip_on=False))
ax4.set_yticks([-2*A,-A,-A/2,-A/4,0,A])
ax4.set_yticklabels(['$-2A\sigma_0$','$-A\sigma_0$',r'$-\faktor{A\sigma_0}{2}$',r'$-\faktor{A\sigma_0}{4}$','$0$','$A$'])
ax4.set_ylim(-2*A*sigma_0,1)
ax4.set_xlim(1,-np.log(2**7)/np.real(s_infty0))
ax4.set_xticks(np.arange(0,-np.log(2**8)/np.real(s_infty0),-np.log(2)/np.real(s_infty0)))
ax4.set_xticklabels(xticklabels)
ax4.grid(True)
ax4.set_title(r'$h(t)=A\delta(t)-2A\sigma_0\e^{-\sigma_0t}\epsilon(t)$',pad=10)
ax4.set_xlabel('$t$')

################################################################################

# s-plane

ax5.fill_between(s2,
                 np.zeros_like(s2)-2,
                 np.zeros_like(s2)+2,
                 color='C2',
                 clip_on=True,
                 label=r'$\mathrm{ROC}\{s\}$',
                 alpha=0.5,
                )
ax5.set_xlabel(r'$\mathrm{Re}\{s\}$')
ax5.set_ylabel(r'$\mathrm{Im}\{s\}$')
ax5.scatter(np.real(s_infty1),np.imag(s_infty1),
            s=75,
            marker='x',
            color='C0'
           )

ax5.scatter(np.real(s_infty2),np.imag(s_infty2),
            s=75,
            marker='x',
            color='C0'
           )
ax5.scatter(np.real(s_zero1),np.imag(s_zero1),
            s=75,
            marker='o',
            ec='C0',
            fc='none'
           )
ax5.set_xlim(-2*sigma_0,2*sigma_0)
ax5.set_ylim(-2*omega_0,2*omega_0)
ax5.set_xticks(np.arange(-2,2+1,1))
ax5.set_yticks(np.arange(-2,2+1,1))
ax5.set_xticklabels([r'$-2\sigma_0$',r'$-\sigma_0$','$0$',r'$+\sigma_0$',r'$+2\sigma_0$'])
ax5.set_yticklabels([r'$-2\omega_0$',r'$-\omega_0$',r'$0$',r'$+\omega_0$',r'$+2\omega_0$'])
ax5.text(1,-1.5,
         s='$g=A$',
         bbox=dict(boxstyle='round',ec='black',fc='white'),
         ha='center')
ax5.grid(True)
ax5.annotate(r'$s_{\infty,1}$',
             (np.real(s_infty1),np.imag(s_infty1)),
             (np.real(s_infty1),np.imag(s_infty1)-0.5),
            size='x-large',
            ha = 'center')
ax5.annotate(r'$s_{\infty,2}$',
             (np.real(s_infty2),np.imag(s_infty2)),
             (np.real(s_infty2),np.imag(s_infty2)-0.5),
            size='x-large',
            ha = 'center')
ax5.annotate(r'$s_{0,1}$',
             (np.real(s_zero1),np.imag(s_zero1)),
             (np.real(s_zero1),np.imag(s_zero1)-0.5),
            size='x-large',
            ha = 'center')
ax5.set_box_aspect(1)
ax5.legend(edgecolor='black')
ax5.set_title(r'$H(s)=A\frac{s+\faktor{\sigma_0}{4}}{(s-(-\faktor{\sigma_0}{4}+\im\omega_0))(s-(-\faktor{\sigma_0}{4}-\im\omega_0))}$',pad=10)

# time-domain

system = signal.lti([s_zero1],[s_infty1,s_infty2],A)
t = np.arange(0,28*np.log(2)/sigma_0+1/1000,1/1000)
t, impulse = signal.lti.impulse(system,T=t)
ax6.plot(t,impulse,clip_on=False)
ax6.set_yticks([-A,-A/2,-A/4,0,A/4,A/2,A])
ax6.set_yticklabels(['$-A$',r'$-\faktor{A}{2}$',r'$-\faktor{A}{4}$','$0$',r'$+\faktor{A}{4}$',r'$+\faktor{A}{2}$','$+A$'])
ax6.set_ylim(-A,A)
ax6.set_xticks(np.arange(0,28*np.log(2)/sigma_0+4*np.log(2)/sigma_0,4*np.log(2)/sigma_0))
xticklabels = [r'$0$']
for i in range(4,28+4,4):
    xticklabels += [r'$\frac{%d\ln(2)}{\sigma_0}$'%(i)]
ax6.set_xticklabels(xticklabels)
ax6.set_xlim(0,28*np.log(2)/sigma_0)
ax6.grid(True)
ax6.set_title(r'$h(t)=\e^{-\frac{\sigma_0}{4}t}\cos\left(\omega t\right)\epsilon(t)$',pad=20)
ax6.set_xlabel('$t$')

################################################################################

fig.tight_layout()
fig.savefig('laplace.pdf')

In [ ]:
z = Symbol('z')
r = Symbol('r')
a = Symbol('a')
b = Symbol('b')
phi = Symbol(r'\varphi')
A = Symbol('A')
Omega_0 = Symbol('Omega_0')
F = (z-2)/(z-Rational(1,2))/z
apart(F,z)*z

In [ ]:
fig, ((ax1,ax2),(ax3,ax4),(ax5,ax6)) = plt.subplots(ncols=2,nrows=3,gridspec_kw=dict(width_ratios=[1,2.5]),figsize=(11,9))

################################################################################
# poles, zeros, gain
A = 1
r = 1/2
Omega_0 = 4*np.log(2)/sigma_0
z_infty0 = r
z_zero0 = 0
#z_zero1 = r+sigma_0
z_zero1 = 1/r
z_zero2 = r*np.cos(Omega_0)
z_infty1 = r*np.exp(1j*Omega_0)
z_infty2 = r*np.exp(-1j*Omega_0)
z = np.arange(-3,3+1/1000,1/1000)

################################################################################


ax1.fill_between(z,
                 np.zeros_like(z)-1.5,
                 np.zeros_like(z)+1.5,
                 color='C2',
                 clip_on=True,
                 label=r'$\mathrm{ROC}\{z\}$',
                 alpha=0.5,
                 zorder=0,
                )

circle = Circle((0,0),
                np.abs(z_infty0),
                fc='white',
                ec='white',
                zorder=0,
                lw=0,
               )
ax1.add_patch(circle)
fc = colors.to_rgba(c='C2',alpha=0)
circle = Circle((0,0),
                1,
                ec='black',
                fc=fc,
                lw=2,
                zorder=1,
               )
ax1.add_patch(circle)
ax1.set_box_aspect(1)

ax1.set_xlabel(r'$\mathrm{Re}\{z\}$')
ax1.set_ylabel(r'$\mathrm{Im}\{z\}$')
ax1.scatter(np.real(z_infty0),np.imag(z_infty0),
            s=75,
            marker='x',
            color='C0',
            zorder=1,
           )
ax1.annotate(r'$z_{\infty,0}$',
             (np.real(z_infty0),np.imag(z_infty0)),
             (np.real(z_infty0),np.imag(z_infty0)-0.25),
                size='x-large',
            ha = 'center',
            )

ax1.scatter(np.real(z_zero0),np.imag(z_zero0),
            s=75,
            marker='o',
            ec='C0',
            fc = 'white',
            zorder=1,
           )
ax1.annotate(r'$z_{0,0}$',
             (np.real(z_zero0),np.imag(z_zero0)),
             (np.real(z_zero0),np.imag(z_zero0)-0.25),
            size='x-large',
            ha = 'center',
            )
ax1.text(0,-0.75,
         s='$g=A$',
         bbox=dict(boxstyle='square',ec='black',fc='white'),
         ha='center')
ax1.set_xticks(np.arange(-1,2,1))
ax1.set_yticks(np.arange(-1,2,1))
ax1.set_xlim(-1,1)
ax1.set_ylim(-1,1)
ax1.legend(loc='upper right')
ax1.set_title(r'$H(z)=A\frac{z}{z-\faktor{1}{2}}$',pad=20)
ax1.grid(True)
ax1.set_axisbelow(True)

system = signal.dlti([z_zero0],[z_infty0],A)
k, impulse = signal.dimpulse(system,n=8)
impulse = impulse[0]

ax2.plot(k,impulse,ls='',marker='o',mfc='C0',mec='C0',clip_on=False)
k = np.arange(0,7+1/1000,1/1000)
impulse2 = z_infty0**k
ax2.plot(k,impulse2,ls=':',color='C0',clip_on=False)


ax2.set_xticks(np.arange(0,8))
ax2.set_yticks([0,A/8,A/4,A/2,A])
ax2.set_yticklabels(['$0$',r'$\faktor{A}{8}$',r'$\faktor{A}{4}$',r'$\faktor{A}{2}$','$A$'])

ax2.set_xlim(0,7)
ax2.set_ylim(0,A)

ax2.set_xlabel('$k$')
ax2.set_title(r'$h[k]=A\left(\frac{1}{2}\right)^k\epsilon[k]$',pad=20)
ax2.grid(True)

################################################################################

# z-plane
ax3.fill_between(z,
                 np.zeros_like(z)-2.5,
                 np.zeros_like(z)+2.5,
                 color='C2',
                 clip_on=True,
                 label=r'$\mathrm{ROC}\{z\}$',
                 alpha=0.5,
                 zorder=0,
                )

circle = Circle((0,0),
                np.abs(z_infty0),
                fc='white',
                ec='white',
                zorder=0,
                lw=0,
               )
ax3.add_patch(circle)
fc = colors.to_rgba(c='C2',alpha=0)
circle = Circle((0,0),
                1,
                ec='black',
                fc=fc,
                lw=2,
                zorder=1,
               )
ax3.add_patch(circle)
ax3.set_box_aspect(1)

ax3.set_xlabel(r'$\mathrm{Re}\{z\}$')
ax3.set_ylabel(r'$\mathrm{Im}\{z\}$')
ax3.scatter(np.real(z_infty0),np.imag(z_infty0),
            s=75,
            marker='x',
            color='C0',
            zorder=1,
           )
ax3.annotate(r'$z_{\infty,0}$',
            (np.real(z_infty0),np.imag(z_infty0)),
            (np.real(z_infty0),np.imag(z_infty0)-0.5),
            size='x-large',
            ha = 'right',
             
            )

ax3.scatter(np.real(z_zero1),np.imag(z_zero1),
            s=75,
            marker='o',
            ec='C0',
            fc = fc,
            zorder=1,
            clip_on=False,
           )
ax3.annotate(r'$z_{0,1}$',
             (np.real(z_zero1),np.imag(z_zero1)),
             (np.real(z_zero1),np.imag(z_zero1)-0.5),
            size='x-large',
            ha = 'center',
            )
ax3.text(0,-0.75*2,
         s='$g=A$',
         bbox=dict(boxstyle='square',ec='black',fc='white'),
         ha='center')
ax3.set_xticks(np.arange(-2,2.5,1))
ax3.set_yticks(np.arange(-2,2.5,1))
ax3.set_xlim(-2,2)
ax3.set_ylim(-2,2)
ax3.legend(loc='upper right')
ax3.set_title(r'$H(z)=A\frac{z-2}{z-\faktor{1}{2}}$',pad=20)
ax3.grid(True)
ax3.set_axisbelow(True)

# time domain

system = signal.dlti([z_zero1],[z_infty0],A)
k, impulse = signal.dimpulse(system,n=8)
impulse = impulse[0]

ax4.plot(k,impulse,ls='',marker='o',mfc='C0',mec='C0',clip_on=False)
k = np.arange(0,7+1/1000,1/1000)
impulse2 = -3*A*(1/2)**k
ax4.plot(k,impulse2,ls=':',color='C0',clip_on=False)

#ax4.set_xticks(np.arange(0,8))
ax4.set_yticks([-3*A,-3*A/2,-3*A/4,-3*A/8,0,A])
ax4.set_yticklabels(['$-3A$',r'$-\faktor{3A}{2}$',r'$-\faktor{3A}{4}$',r'$-\faktor{3A}{8}$','$0$','$A$'])

ax4.set_xlim(0,7)
ax4.set_ylim(-3*A,A)

ax4.set_title(r'$h[k]=4A\delta[k]-3A\left(\frac{1}{2}\right)^k\epsilon[k]$',pad=20)
ax4.set_xlabel('$k$')
ax4.grid(True)

################################################################################

"""
ax3.fill_between(z,
                 np.zeros_like(z)-2.5,
                 np.zeros_like(z)+2.5,
                 color='C2',
                 clip_on=True,
                 label=r'$\mathrm{ROC}\{z\}$',
                 alpha=0.5,
                 zorder=0,
                )

circle = Circle((0,0),
                np.abs(z_infty0),
                fc='white',
                ec='white',
                zorder=0,
                lw=0,
               )
ax3.add_patch(circle)
fc = colors.to_rgba(c='C2',alpha=0)
circle = Circle((0,0),
                1,
                ec='black',
                fc=fc,
                lw=2,
                zorder=1,
               )
ax3.add_patch(circle)
ax3.set_box_aspect(1)

ax3.set_xlabel(r'$\mathrm{Re}\{z\}$')
ax3.set_ylabel(r'$\mathrm{Im}\{z\}$')
ax3.scatter(np.real(z_infty0),np.imag(z_infty0),
            s=75,
            marker='x',
            color='C0',
            zorder=1,
           )
ax3.annotate(r'$z_{\infty,0}$',
            (np.real(z_infty0),np.imag(z_infty0)),
            (np.real(z_infty0),np.imag(z_infty0)-0.5),
            size='x-large',
            ha = 'right',
             
            )

ax3.scatter(np.real(z_zero1),np.imag(z_zero1),
            s=75,
            marker='o',
            ec='C0',
            fc = fc,
            zorder=1,
           )
ax3.annotate(r'$z_{0,1}$',
             (np.real(z_zero1),np.imag(z_zero1)),
             (np.real(z_zero1),np.imag(z_zero1)-0.5),
            size='x-large',
            ha = 'center',
            )
ax3.text(0,-0.75*2,
         s='$g=A$',
         bbox=dict(boxstyle='square',ec='black',fc='white'),
         ha='center')
ax3.set_xticks(np.arange(-2,2.5,1))
ax3.set_yticks(np.arange(-2,2.5,1))
ax3.set_xlim(-2,2)
ax3.set_ylim(-2,2)
ax3.legend(loc='upper right')
ax3.set_title(r'$H(z)=A\frac{z-\left(\faktor{1}{2}+\sigma_0\right)}{z-\faktor{1}{2}}$',pad=20)
ax3.grid(True)
ax3.set_axisbelow(True)

system = signal.dlti([z_zero1],[z_infty0],A)
k, impulse = signal.dimpulse(system,n=8)
impulse = impulse[0]

ax4.plot(k,impulse,ls='',marker='o',mfc='C0',mec='C0',clip_on=False)
k = np.arange(0,7+1/1000,1/1000)
impulse2 = -2*A*sigma_0*(1/2)**k
ax4.plot(k,impulse2,ls=':',color='C0',clip_on=False)
ax4.annotate(text='',xy=(0,0),xytext=(0,A),
             arrowprops=dict(arrowstyle='<-',
                             connectionstyle='arc3',
                             shrinkA=0,shrinkB=0,
                             ec='C0',fc='C0',
                             lw=1.5,
                             ls=':',
                             clip_on=False))

#ax4.set_xticks(np.arange(0,8))
ax4.set_yticks([-2*A*sigma_0,-A*sigma_0,-A/2*sigma_0,-A/4*sigma_0,0,A])
ax4.set_yticklabels(['$-2A\sigma_0$','$-A\sigma_0$',r'$-\faktor{A\sigma_0}{2}$',r'$-\faktor{A\sigma_0}{4}$','$0$','$A$'])

ax4.set_xlim(0,7)
ax4.set_ylim(-2*A*sigma_0,A)

ax4.set_title(r'$h[k]=A\delta[k]-2A\sigma_0\left(\frac{1}{2}\right)\epsilon[k]$',pad=20)
ax4.set_xlabel('$k$')
ax4.grid(True)
"""

################################################################################

# z-plane

ax5.fill_between(z,
                 np.zeros_like(z)-2.5,
                 np.zeros_like(z)+2.5,
                 color='C2',
                 clip_on=True,
                 label=r'$\mathrm{ROC}\{z\}$',
                 alpha=0.5,
                 zorder=0,
                )

circle = Circle((0,0),
                np.abs(z_infty0),
                fc='white',
                ec='white',
                zorder=0,
                lw=0,
               )
ax5.add_patch(circle)
fc = colors.to_rgba(c='C2',alpha=0)
circle = Circle((0,0),
                1,
                ec='black',
                fc=fc,
                lw=2,
                zorder=1,
               )
ax5.add_patch(circle)
ax5.set_box_aspect(1)

ax5.set_xlabel(r'$\mathrm{Re}\{z\}$')
ax5.set_ylabel(r'$\mathrm{Im}\{z\}$')
ax5.scatter(np.real(z_infty1),np.imag(z_infty1),
            s=75,
            marker='x',
            color='C0',
            zorder=1,
           )
ax5.annotate(r'$z_{\infty,1}$',
            (np.real(z_infty1),np.imag(z_infty1)),
            (np.real(z_infty1),np.imag(z_infty1)+0.3),
            size='x-large',
            ha = 'center',
             va = 'center',
             
            )
ax5.scatter(np.real(z_infty2),np.imag(z_infty2),
            s=75,
            marker='x',
            color='C0',
            zorder=1,
           )
ax5.annotate(r'$z_{\infty,2}$',
            (np.real(z_infty2),np.imag(z_infty2)),
            (np.real(z_infty2),np.imag(z_infty2)-0.3),
            size='x-large',
            ha = 'center',
             
            )
ax5.scatter(np.real(z_zero2),np.imag(z_zero2),
            s=75,
            marker='o',
            ec='C0',
            fc = fc,
            zorder=1,
           )
ax5.annotate(r'$z_{0,1}$',
             (np.real(z_zero2),np.imag(z_zero2)),
             (np.real(z_zero2)-0.1,np.imag(z_zero2)),
            size='x-large',
            ha = 'right',
            va = 'center',
            )
ax5.scatter(np.real(z_zero0),np.imag(z_zero0),
            s=75,
            marker='o',
            ec='C0',
            fc = fc,
            zorder=1,
           )
ax5.annotate(r'$z_{0,0}$',
             (np.real(z_zero0),np.imag(z_zero0)),
             (np.real(z_zero0),np.imag(z_zero0)-0.25),
            size='x-large',
            ha = 'center',
            va = 'center',
            )
ax5.text(0,-0.75,
         s='$g=A$',
         bbox=dict(boxstyle='square',ec='black',fc='white'),
         ha='center')
ax5.set_xticks(np.arange(-1,1+1,1))
ax5.set_yticks(np.arange(-1,1+1,1))
ax5.set_xlim(-1,1)
ax5.set_ylim(-1,1)
ax5.legend(loc='upper right')
ax5.set_title(r'$H(z)=A\frac{z\left(z-\faktor{1}{2}\cos(\Omega_0\right)}{\left(z-\faktor{1}{2}\e^{+\im\Omega_0}\right)\left(z-\faktor{1}{2}\e^{-\im\Omega_0}\right)}$',pad=20)
ax5.grid(True)
ax5.set_axisbelow(True)

# time domain

system = signal.dlti([z_zero0,z_zero2],[z_infty1,z_infty2],A)
k, impulse = signal.dimpulse(system,n=8)
impulse = impulse[0]

ax6.plot(k,impulse,ls='',marker='o',mfc='C0',mec='C0',clip_on=False)

k = np.arange(0,7+1/1000,1/1000)
h = r**k*np.cos(Omega_0*k)
ax6.plot(k,h,ls=':',color='C0',clip_on=False)
ax6.set_xticks(np.arange(8))
ax6.set_yticks([-A,-A/2,-A/4,0,A/4,A/2,A])
ax6.set_yticklabels(['$-A$',r'$- \faktor{A}{2}$',r'$-\faktor{A}{4}$','$0$',r'$+\faktor{A}{4}$',r'$+\faktor{A}{2}$','$+A$'])

ax6.set_xlim(0,7)
ax6.set_ylim(-A,A)
ax6.set_title(r'$h[k]=\left(\frac{1}{2}\right)^k\cos[\Omega_0 k]\epsilon[k]$',pad=20)
ax6.set_xlabel('$k$')
ax6.grid(True)

################################################################################

fig.tight_layout()
fig.savefig('zTrafo.pdf')